#**IRWA 2024 - PROJECT**
#**Part 3: Ranking**
##**G102_2**

Roger Sola (u199780)

Queralt Zamora (u199903)

Clàudia Morales (u199906)




#**Initialize**

In [201]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [202]:
# TO BE MODIFIED WITH THE DIRECTORY FROM THE TEACHER

#docs_path = 'farmers-protest-tweets.json'
docs_path = '/content/drive/MyDrive/1st TERM/RIAW/LABS/data/farmers-protest-tweets.json' #queralt
#docs_path = '/content/drive/MyDrive/UNI/UPF/4t Any/RIAW/LABS/data/farmers-protest-tweets.json'#claud
#docs_path = '/content/drive/MyDrive/4t/T1/RIAW/LABS/data/farmers-protest-tweets.json' #roger

In [203]:
# TO BE MODIFIED WITH THE DIRECTORY FROM THE TEACHER

#ids_path = 'tweet_document_ids_map.csv'
ids_path = '/content/drive/MyDrive/1st TERM/RIAW/LABS/data/tweet_document_ids_map.csv' #queralt
#ids_path  = '/content/drive/MyDrive/UNI/UPF/4t Any/RIAW/LABS/data/tweet_document_ids_map.csv' #claudia
#ids_path  = '/content/drive/MyDrive/4t/T1/RIAW/LABS/data/tweet_document_ids_map.csv' #roger

In [204]:
# TO BE MODIFIED WITH THE DIRECTORY FROM THE TEACHER

evaluation_path = '/content/drive/MyDrive/1st TERM/RIAW/LABS/data/evaluation_gt.csv' #queralt
#evaluation_path = '/content/drive/MyDrive/UNI/UPF/4t Any/RIAW/LABS/data/evaluation_gt.csv' #claudia
#evaluation_path = '/content/drive/MyDrive/4t/T1/RIAW/LABS/data/evaluation_gt.csv' #roger

In [205]:
import json
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from string import punctuation
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from collections import Counter
from wordcloud import WordCloud
import string
import spacy
from IPython.display import Markdown, display
import numpy as np
from collections import defaultdict
import time
from array import array
import collections
from numpy import linalg as la
import math
import random
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.manifold import TSNE
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import Word2Vec
np.random.seed(42)
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

#**PART 1: Preprocessing Data and Analysis**

##**Preprocessing Data**

<font size="+2">2. Preprocessing Data</font>

1.  *Removing stop words*
1.  *Tokenization*
2.  *Removing punctuation marks*
1.  *Stemming*
2.  *and... anything else you think it's needed (bonus point)*

*HINTS:*

*1. Take into account that for future queries, the final output must return (when present)
the following information for each of the selected documents: Tweet | Date | Hashtags
| Likes | Retweets | Url (here the “Url” means the tweet link).*

*2. Consider your approach to hashtags during pre-processing, such as the decision to retain
or remove the "#" symbol. Their distinctiveness might be valuable when treated as
separate terms in the inverted index. As guidance, turn to the evaluation file
(evaluation_gt) which will become pivotal in the project’s second phase. Using a subset
of the dataset, the evaluation_gt sets a baseline with three distinct information needs
and their respective ground truths, indicating if a document is relevant (1) or not (0) to
an information need. For context, one of these information needs relates to discussions
about Indian goverment. Reflect on how this context might shape your strategy for
hashtag handling.*

*3. Suggested library that may help you in stemming and stop words: nltk*


*Make sure you map the tweet’s Ids with the document ids as the document Ids will be
considered for the evaluation stage of the project (tweet_document_ids_map).*

In [206]:
def create_markdown(title):
    display(Markdown(f'### {title}'))

nltk.download('punkt')
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


<font size="+1">2.1. - 2.5. Preprocessesing function
</font>

In [207]:
# preprocessing function
def preprocess_text(text):
    text = text.lower() # 2.1. convert text to lowercase
    text = re.sub(r'http\S+|www\S+', '', text) # 2.2. remove URLs
    hashtags = re.findall(r'#\S+', text) # 2.3. extract hashtags separately (HINT 2)
    text = re.sub(r'#\w+', '', text) # 2.3. delete hastags from main tweet (HINT 2)
    # more preprocess methods
    text = text.strip() # 2.4. removing spaces at the beginning and spaces at the end
    text = re.sub(r'\s+', ' ', text) # 2.4. changing double spaces to single spaces.

    # NEW: we eliminate everything that is not alphanumeric characters
    text = re.sub(r'[^a-zA-Z0-9\s]+', '', text)

    tokens = word_tokenize(text) # 2.5. tokenize text
    filtered_tokens = [stemmer.stem(word) for word in tokens if word not in stop_words and word not in punctuation ] # remove punctuation and stop words, and apply stemming

    return filtered_tokens, hashtags

<font size="+1">2.6. Preprocessed Tweets storage
</font>

In [208]:
# 2.6. read and preprocess the data line by line from .json file
preprocessed_tweets = []
with open(docs_path, 'r') as file:
    for line in file:
        try:
            tweet = json.loads(line)

            # extract relevant information
            tweet_id = tweet.get('id')
            content = tweet.get('content')
            preprocessed_content,hashtags = preprocess_text(content)
            preprocessed_tweets.append({
                'id': tweet_id,
                'preprocessed_content': preprocessed_content,
                'date': tweet.get('date'),
                'hashtags': hashtags,
                'likes': tweet.get('likeCount'),
                'retweets': tweet.get('retweetCount'),
                'url': tweet.get('url')
            })
        except json.JSONDecodeError as e:
            print(f"Error parsing line: {e}")

In [209]:
# function to contain all necessary tweet and document information
def return_tweet_info(preprocessed_tweets):

    df = pd.DataFrame(preprocessed_tweets) # create DataFrame

    # new dataframe with desired information
    tweet_info = pd.DataFrame({
        'Tweet ID': df['id'],
        'Tweet': df['preprocessed_content'].apply(lambda x: ' '.join(x)),
        'Date': df['date'],
        'Hashtags': df['hashtags'].apply(lambda x: ', '.join(x) if x else 'None'),
        'Likes': df['likes'],
        'Retweets': df['retweets'],
        'Url': df['url']
    })

    return tweet_info

In [210]:
tweet_info_df = return_tweet_info(preprocessed_tweets)
display(tweet_info_df)

,Tweet ID,Tweet,Date,Hashtags,Likes,Retweets,Url
0,1364506249291784198,world progress indian polic govt still tri tak...,2021-02-24T09:23:35+00:00,"#modidontsellfarmers, #farmersprotest, #freeno...",0,0,https://twitter.com/ArjunSinghPanam/status/136...
1,1364506237451313155,kisanektamorcha farmer constantli distroy crop...,2021-02-24T09:23:32+00:00,"#farmersprotest, #modiignoringfarmersdeaths, #...",0,0,https://twitter.com/PrdeepNain/status/13645062...
2,1364506195453767680,,2021-02-24T09:23:22+00:00,#farmersprotest,0,0,https://twitter.com/parmarmaninder/status/1364...
3,1364506167226032128,reallyswara rohinisgh watch full video,2021-02-24T09:23:16+00:00,"#farmersprotest, #nofarmersnofood",0,0,https://twitter.com/anmoldhaliwal/status/13645...
4,1364506144002088963,,2021-02-24T09:23:10+00:00,"#kisanektamorcha, #farmersprotest, #nofarmersn...",0,0,https://twitter.com/KotiaPreet/status/13645061...
...,...,...,...,...,...,...,...
117402,1360040182771163138,,2021-02-12T01:37:02+00:00,"#farmersprotest, #kisanandolan, #kisaanmajdoor...",0,0,https://twitter.com/rickyrickstir/status/13600...
117403,1360040146402373637,pm punjabtak,2021-02-12T01:36:53+00:00,"#farmersprotest, #kisanaandolan, #farmersfury,...",0,0,https://twitter.com/PunjabTak/status/136004014...
117404,1360040134230556678,unit stand divid fall,2021-02-12T01:36:50+00:00,"#mahapanchayatrevolution, #farmersprotest",39,65,https://twitter.com/ish_kayy/status/1360040134...
117405,1360040127679000577,,2021-02-12T01:36:49+00:00,"#farmersprotest, #msp, #farmbill2020, #singhub...",15,1,https://twitter.com/TV9Bharatvarsh/status/1360...


<font size="+1">2.7. Map IDs</font>

In [211]:
# 2.7. map the tweet’s ids with the document ids
# ids csv loaded
tweet_ids_map = pd.read_csv(ids_path)
display(tweet_ids_map)

,docId,id
0,doc_0,1364506249291784198
1,doc_1,1364506237451313155
2,doc_2,1364506167226032128
3,doc_3,1364505991887347714
4,doc_4,1364505813834989568
...,...,...
48424,doc_48424,1360040232733708294
48425,doc_48425,1360040229265022979
48426,doc_48426,1360040195786067969
48427,doc_48427,1360040134230556678


In [212]:
# merged the tweet_info_df with the tweet_ids_map based on the 'Tweet ID' column
# 'id' in tweet_ids_map corresponds to 'Tweet ID' in tweet_info_df
tweet_info_df = tweet_info_df.merge(tweet_ids_map, left_on='Tweet ID', right_on='id', how='left')

# renamed 'docid' column from tweet_ids_map to 'Document ID' in the merged DataFrame
tweet_info_df = tweet_info_df.rename(columns={'docId': 'Document ID'})

# drop the column 'id' to not have repeated columns
tweet_info_df = tweet_info_df.drop(columns=['id'])

# NEW: drop the rows without doc_id
tweet_info_df = tweet_info_df.dropna(subset=['Document ID'])

display(tweet_info_df)

,Tweet ID,Tweet,Date,Hashtags,Likes,Retweets,Url,Document ID
0,1364506249291784198,world progress indian polic govt still tri tak...,2021-02-24T09:23:35+00:00,"#modidontsellfarmers, #farmersprotest, #freeno...",0,0,https://twitter.com/ArjunSinghPanam/status/136...,doc_0
1,1364506237451313155,kisanektamorcha farmer constantli distroy crop...,2021-02-24T09:23:32+00:00,"#farmersprotest, #modiignoringfarmersdeaths, #...",0,0,https://twitter.com/PrdeepNain/status/13645062...,doc_1
3,1364506167226032128,reallyswara rohinisgh watch full video,2021-02-24T09:23:16+00:00,"#farmersprotest, #nofarmersnofood",0,0,https://twitter.com/anmoldhaliwal/status/13645...,doc_2
8,1364505991887347714,mandeeppunia1 watch full video,2021-02-24T09:22:34+00:00,"#farmersprotest, #nofarmersnofood",0,0,https://twitter.com/anmoldhaliwal/status/13645...,doc_3
11,1364505813834989568,mandeeppunia1 watch full video,2021-02-24T09:21:51+00:00,"#farmersprotest, #nofarmersnofood",0,0,https://twitter.com/anmoldhaliwal/status/13645...,doc_4
...,...,...,...,...,...,...,...,...
117395,1360040232733708294,modi want make slave free india come join farm...,2021-02-12T01:37:14+00:00,"#mahapanchayatrevolution, #farmersprotest",208,246,https://twitter.com/AntiITcellTask1/status/136...,doc_48424
117397,1360040229265022979,farmer agit longer confin border one state far...,2021-02-12T01:37:13+00:00,"#mahapanchayatrevolution, #farmersprotest",22,31,https://twitter.com/rupindr79/status/136004022...,doc_48425
117400,1360040195786067969,first mahapanchayat punjab revolut gain strength,2021-02-12T01:37:05+00:00,"#mahapanchayatrevolution, #farmersprotest",18,43,https://twitter.com/PushpSamra/status/13600401...,doc_48426
117404,1360040134230556678,unit stand divid fall,2021-02-12T01:36:50+00:00,"#mahapanchayatrevolution, #farmersprotest",39,65,https://twitter.com/ish_kayy/status/1360040134...,doc_48427


In [213]:
print('There are a total of',len(tweet_info_df),'tweets')

There are a total of 48429 tweets


In [214]:
# verify the number of entries maped
print(f"Number of entries maped: {len(tweet_ids_map)}")

Number of entries maped: 48429


<font size="+1">2.8. Output display</font>

In [215]:
# 2.8. printing method as demanded (HINT 1)
print("Tweet | Date | Hashtags | Likes | Retweets | Url")
print()
for index, row in tweet_info_df.head().iterrows():
    print(index, f"| {row['Tweet']} | {row['Date']} | {row['Hashtags']} | {row['Likes']} | {row['Retweets']} | {row['Url']}")

# print one of the most retweeted tweets to see if the variables of the number of likes/retweets are correctly created.
most_retweeted_tweet = tweet_info_df.iloc[111]
print(f"{most_retweeted_tweet.name} | {most_retweeted_tweet['Tweet']} | {most_retweeted_tweet['Date']} | {most_retweeted_tweet['Hashtags']} | {most_retweeted_tweet['Likes']} | {most_retweeted_tweet['Retweets']} | {most_retweeted_tweet['Url']}")

Tweet | Date | Hashtags | Likes | Retweets | Url

0 | world progress indian polic govt still tri take india back horrif past tyranni narendramodi delhipolic shame | 2021-02-24T09:23:35+00:00 | #modidontsellfarmers, #farmersprotest, #freenodeepkaur | 0 | 0 | https://twitter.com/ArjunSinghPanam/status/1364506249291784198
1 | kisanektamorcha farmer constantli distroy crop throughout india realli heart breakingw care crop like children govt agricultur minist laugh usw win | 2021-02-24T09:23:32+00:00 | #farmersprotest, #modiignoringfarmersdeaths, #modidontsellfarmers | 0 | 0 | https://twitter.com/PrdeepNain/status/1364506237451313155
3 | reallyswara rohinisgh watch full video | 2021-02-24T09:23:16+00:00 | #farmersprotest, #nofarmersnofood | 0 | 0 | https://twitter.com/anmoldhaliwal/status/1364506167226032128
8 | mandeeppunia1 watch full video | 2021-02-24T09:22:34+00:00 | #farmersprotest, #nofarmersnofood | 0 | 0 | https://twitter.com/anmoldhaliwal/status/1364505991887347714
11 | mandeeppun

It was decided to not take out the emogis since then, the special characters in indian would be also be deleted, leaving some tweets in blank.

##**Data analysis**

<font size="+2">3. Data Analysis</font>

*When working with data, it is important to have a better understanding of the content and
some statistics. Provide an exploratory data analysis to describe the dataset you are working on
in this project and explain the decisions made for the analysis. For example, word counting
distribution, average sentence length, vocabulary size, ranking of tweets most retweeted, word
clouds for the most frequent words, and entity recognition. Feel free to do the exploratory
analysis and report your findings in the report.*

<font size="+1">3.1. Word count distribution</font>

In [216]:
# 3.1. Word count distribution
word_count = tweet_info_df['Tweet'].apply(lambda x: len(x.split())) # num of words per tweet
'''
# create_markdown("1. Word count distribution")
plt.figure(figsize=(10, 6))
sns.histplot(word_count, kde=True, bins=30, color = 'purple')
plt.title('Word Count Distribution Across Tweets')
plt.xlabel('Number of Words')
plt.ylabel('Frequency')
plt.show()
'''

'\n# create_markdown("1. Word count distribution")\nplt.figure(figsize=(10, 6))\nsns.histplot(word_count, kde=True, bins=30, color = \'purple\')\nplt.title(\'Word Count Distribution Across Tweets\')\nplt.xlabel(\'Number of Words\')\nplt.ylabel(\'Frequency\')\nplt.show()\n'

The number of words in tweets seems to follow a geometric (or exponential) distribution. Therefore, we can say that tweets tend to be short texts. In fact, there is a peak at the beginning, indicating that most of them do not exceed 5 to 10 words.

<font size="+1">3.2. Average sentence length</font>

In [217]:
# 3.2. Average sentence length

#create_markdown("2. Average sentence length")
average_sentence_length = word_count.mean()
print(f"Average number of words per tweet: {average_sentence_length:.2f}")

Average number of words per tweet: 11.26


We calculated the average sentence length across all tweets to provide how much the users tend to write in their tweets. In the result, we can see that the average tweet length contains around 10 words which makes sense observing the previous section. This tells us that most users are concise at the moment to write their tweets.

<font size="+1">3.3. Vocabulary size</font>

In [218]:
# 3.3. Vocabulary size

#create_markdown("3. Vocabulary size")
all_words = ' '.join(tweet_info_df['Tweet']).split() # combine all words from the tweets
vocabulary = set(all_words)
vocab_size = len(vocabulary)# Count unique words (vocabulary size)
print(f"Vocabulary size (unique words): {vocab_size}")

Vocabulary size (unique words): 32246


To observe the diversity of the language used in the dataset we calculated the vocabulary size, the number of unique words across all tweets. We get as a result a total of 104894 unique words, indicating a rich and diverse use of the language.

<font size="+1">3.4. Ranking of tweets based on Likes and Retweets</font>

In [219]:
# 3.4. Ranking of tweets based on Likes and Retweets
'''
create_markdown("Ranking of tweets based on Likes and Retweets")

top_retweeted = tweet_info_df[['Tweet', 'Retweets', 'Url']].sort_values(by='Retweets', ascending=False).head(10)
create_markdown("Top 10 Most Retweeted Tweets:")
display(top_retweeted)

top_liked = tweet_info_df[['Tweet', 'Likes', 'Url']].sort_values(by='Likes', ascending=False).head(10)
create_markdown("Top 10 Liked Tweets:")
display(top_liked)
'''

'\ncreate_markdown("Ranking of tweets based on Likes and Retweets")\n\ntop_retweeted = tweet_info_df[[\'Tweet\', \'Retweets\', \'Url\']].sort_values(by=\'Retweets\', ascending=False).head(10)\ncreate_markdown("Top 10 Most Retweeted Tweets:")\ndisplay(top_retweeted)\n\ntop_liked = tweet_info_df[[\'Tweet\', \'Likes\', \'Url\']].sort_values(by=\'Likes\', ascending=False).head(10)\ncreate_markdown("Top 10 Liked Tweets:")\ndisplay(top_liked)\n'

Analysing the previous results, we can see that 7 out of 10 in each top list coincide. Therefore, a tweet with many retweets is likely to have more likes and a tweet with many likes is likely to have more retweets.

<font size="+1"> 3.5. WordCloud of the most frequent words</font>

In [220]:
# 3.5. WordCloud of the most frequent words
'''
#create_markdown("WordCloud of the most frequent words")

# filter out stop words and punctuation, and count word frequencies
word_frequencies = Counter(
    word.strip(string.punctuation).lower() for word in all_words
    if word.lower() not in stop_words and word.strip(string.punctuation) != '' and word.isalpha() #special characters and punctuations out
)

# generate the word cloud
wordcloud = WordCloud(width=800, height=400,colormap='magma_r', background_color='white').generate_from_frequencies(word_frequencies)
plt.figure(figsize=(10, 6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Most Frequent Words in Tweets')
plt.show()
'''

'\n#create_markdown("WordCloud of the most frequent words")\n\n# filter out stop words and punctuation, and count word frequencies\nword_frequencies = Counter(\n    word.strip(string.punctuation).lower() for word in all_words\n    if word.lower() not in stop_words and word.strip(string.punctuation) != \'\' and word.isalpha() #special characters and punctuations out\n)\n\n# generate the word cloud\nwordcloud = WordCloud(width=800, height=400,colormap=\'magma_r\', background_color=\'white\').generate_from_frequencies(word_frequencies)\nplt.figure(figsize=(10, 6))\nplt.imshow(wordcloud, interpolation=\'bilinear\')\nplt.axis(\'off\')\nplt.title(\'Most Frequent Words in Tweets\')\nplt.show()\n'

The most frequent words make quite a lot of sense if we consider the main topic of these tweets: "Farmers Protests of 2021." Therefore, we can see that 'farmer' is the most significant word, apart from 'support', 'protest', and 'india'. Additionally, it could be deduced that these protests took place in India, if it was the case that we did not have this information.

<font size="+1">3.6. Hashtags Analysis</font>

In [221]:
# 3.6. Hashtags Analysis
'''
create_markdown("Top 10 Most common hashtags")
all_hashtags = []

# iterate over each row in the 'Hashtags' column
for hashtags in tweet_info_df['Hashtags']:

    # extract hashtags that start with '#' and stop before a space/comma
    extracted_hashtags = re.findall(r'#\S+[^,\s]', hashtags)
    all_hashtags.extend(extracted_hashtags)

# count the frequency of each hashtag
hashtag_counts = Counter(all_hashtags)

hashtag_df = pd.DataFrame(hashtag_counts.items(), columns=['Hashtag', 'Count'])
hashtag_df = hashtag_df.sort_values(by='Count', ascending=False).reset_index(drop=True)
display(hashtag_df.head(10))
'''

'\ncreate_markdown("Top 10 Most common hashtags")\nall_hashtags = []\n\n# iterate over each row in the \'Hashtags\' column\nfor hashtags in tweet_info_df[\'Hashtags\']:\n\n    # extract hashtags that start with \'#\' and stop before a space/comma\n    extracted_hashtags = re.findall(r\'#\\S+[^,\\s]\', hashtags)\n    all_hashtags.extend(extracted_hashtags)\n\n# count the frequency of each hashtag\nhashtag_counts = Counter(all_hashtags)\n\nhashtag_df = pd.DataFrame(hashtag_counts.items(), columns=[\'Hashtag\', \'Count\'])\nhashtag_df = hashtag_df.sort_values(by=\'Count\', ascending=False).reset_index(drop=True)\ndisplay(hashtag_df.head(10))\n'

In [222]:
'''
top_hashtags = hashtag_counts.most_common(10)
labels, values = zip(*top_hashtags)

plt.figure(figsize=(15, 6))

# plot 1
plt.subplot(1, 2, 1)  # (rows, columns, panel number)
plt.bar(labels, values, color='pink')
plt.title('Top 10 Most Common Hashtags')
plt.xlabel('Hashtags')
plt.ylabel('Frequency')
plt.xticks(rotation=70)

# plot 2
plt.subplot(1, 2, 2)  # (rows, columns, panel number)
log_values = np.log1p(values)  # Using log1p for stability (log(1 + x))
plt.bar(labels, log_values, color='pink')
plt.title('Top 10 Most Common Hashtags (Log Scale)')
plt.xlabel('Hashtags')
plt.ylabel('Log(Frequency)')
plt.xticks(rotation=70)

plt.tight_layout()  # Adjust layout to prevent overlap
plt.show()
'''

"\ntop_hashtags = hashtag_counts.most_common(10)\nlabels, values = zip(*top_hashtags)\n\nplt.figure(figsize=(15, 6))\n\n# plot 1\nplt.subplot(1, 2, 1)  # (rows, columns, panel number)\nplt.bar(labels, values, color='pink')\nplt.title('Top 10 Most Common Hashtags')\nplt.xlabel('Hashtags')\nplt.ylabel('Frequency')\nplt.xticks(rotation=70)\n\n# plot 2\nplt.subplot(1, 2, 2)  # (rows, columns, panel number)\nlog_values = np.log1p(values)  # Using log1p for stability (log(1 + x))\nplt.bar(labels, log_values, color='pink')\nplt.title('Top 10 Most Common Hashtags (Log Scale)')\nplt.xlabel('Hashtags')\nplt.ylabel('Log(Frequency)')\nplt.xticks(rotation=70)\n\nplt.tight_layout()  # Adjust layout to prevent overlap\nplt.show()\n"

In this sixth section, the table and plot reveal that the hashtag #FarmersProtest stands out as the most popular, significantly outpacing the others. However, due to the scale used in the plot, it is really difficult to see the differences in popularity among the remaining hashtags. So we applied a logarithmic scale to enhance visual clarity and make the differences more clear.

<font size="+1">3.7. Relation between number of likes and number of hashtags
</font>

It is believed that tweets with more hashtags tend to receive more likes. In other words, a higher number of hashtags is associated with a greater number of likes.

In [223]:
# 3.7. Relation between number of likes and number of hashtags
'''
def count_hashtags(text):
    return len(re.findall(r'#\S+[^,\s]', text))

hashtag_counts = tweet_info_df['Hashtags'].apply(count_hashtags)

likes = tweet_info_df['Likes']

plt.figure(figsize=(12, 6))
plt.scatter(hashtag_counts, likes, alpha=0.5, color='purple')
plt.title('Comparison of number Likes and number of Hashtags')
plt.xlabel('number of Hashtags')
plt.ylabel('number of Likes')

z = np.polyfit(hashtag_counts, likes, 1)
p = np.poly1d(z)
plt.plot(hashtag_counts, p(hashtag_counts), color='red', linestyle='--')

plt.grid()
plt.show()
'''

"\ndef count_hashtags(text):\n    return len(re.findall(r'#\\S+[^,\\s]', text))\n\nhashtag_counts = tweet_info_df['Hashtags'].apply(count_hashtags)\n\nlikes = tweet_info_df['Likes']\n\nplt.figure(figsize=(12, 6))\nplt.scatter(hashtag_counts, likes, alpha=0.5, color='purple')\nplt.title('Comparison of number Likes and number of Hashtags')\nplt.xlabel('number of Hashtags')\nplt.ylabel('number of Likes')\n\nz = np.polyfit(hashtag_counts, likes, 1)\np = np.poly1d(z)\nplt.plot(hashtag_counts, p(hashtag_counts), color='red', linestyle='--')\n\nplt.grid()\nplt.show()\n"

As observed, our initial expectation was incorrect. Tweets with more likes tend to have fewer hashtags, and the number of likes decreases as the number of hashtags increases.

#**PART 2: Indexing and Evaluation**


<font size="+2">2. Indexing</font>

**2.1. Build inverted index:** *After having pre-processed the data, you can then create the inverted index.*

**HINT** *- you may use the vocabulary data structure, like the one seen during the Practical Labs:*

 *{ Term_id_1: [document_1, document_2, document_4],*

 *Term_id_2: [document_1, document_3, document_5, document_6],
 etc… }*

*Documents information: Since we are dealing with **conjunctive queries (AND)**, each of the returned documents should contain all the words in the query.*

In [224]:
'''# function extracted from professors preparation file

def build_terms(line):
    """
    Preprocess the text by removing stop words, stemming, and tokenizing.

    Arguments:
    line -- string (text) to be preprocessed

    Returns:
    A list of tokens after preprocessing
    """
    stemmer = PorterStemmer()
    stop_words = set(stopwords.words("english"))
    line = line.lower()
    line = re.findall(r'\b\w+\b', line)  # Tokenize the text
    line = [word for word in line if word not in stop_words]  # Remove stopwords
    line = [stemmer.stem(word) for word in line]  # Perform stemming
    return line

# function extracted from professors preparation file with our interested changes applied

def create_index(lines):
    """
    Implement the inverted index.

    Arguments:
    lines -- Collection of documents, each line containing a 'Doc_ID' and the text (tweet)

    Returns:
    index - the inverted index containing terms as keys and the corresponding list of documents and positions as values.
    title_index - mapping of document IDs to titles (tweet texts)
    """
    index = defaultdict(list)
    title_index = {}  # Dictionary to map Doc_ID to tweet text

    for line in lines:  # Lines contain all documents from tweet_info_df
        doc_id, tweet_text = line

        # Store the tweet text in the title_index
        title_index[doc_id] = tweet_text

        # Preprocess the tweet to get terms
        terms = build_terms(tweet_text)

        # Create the current page index for the tweet
        current_page_index = {}

        for position, term in enumerate(terms):
            if term in current_page_index:
                # If term already exists, append the position
                current_page_index[term][1].append(position)
            else:
                # If term is new, create an entry with the doc_id and position list
                current_page_index[term] = [doc_id, [position]]

        # Merge the current page index with the main index
        for term, posting in current_page_index.items():
            index[term].append(posting)

    return index, title_index


lines = list(zip(tweet_info_df['Document ID'], tweet_info_df['Tweet']))

# create the index and title mapping
inverted_index, title_index = create_index(lines)'''

'# function extracted from professors preparation file\n\ndef build_terms(line):\n    """\n    Preprocess the text by removing stop words, stemming, and tokenizing.\n\n    Arguments:\n    line -- string (text) to be preprocessed\n\n    Returns:\n    A list of tokens after preprocessing\n    """\n    stemmer = PorterStemmer()\n    stop_words = set(stopwords.words("english"))\n    line = line.lower()\n    line = re.findall(r\'\x08\\w+\x08\', line)  # Tokenize the text\n    line = [word for word in line if word not in stop_words]  # Remove stopwords\n    line = [stemmer.stem(word) for word in line]  # Perform stemming\n    return line\n\n# function extracted from professors preparation file with our interested changes applied\n\ndef create_index(lines):\n    """\n    Implement the inverted index.\n\n    Arguments:\n    lines -- Collection of documents, each line containing a \'Doc_ID\' and the text (tweet)\n\n    Returns:\n    index - the inverted index containing terms as keys and the co

In [225]:
'''# display a sample from the inverted index and title index
# print("Sample Inverted Index:", {k: inverted_index[k] for k in list(inverted_index)[:5]})
# print("Sample Title Index:", {k: title_index[k] for k in list(title_index)[:5]})
# Print the entire inverted index with one term per row
for term, postings in list(inverted_index.items())[:10]:
    print(f"{term}: {postings}")'''

'# display a sample from the inverted index and title index\n# print("Sample Inverted Index:", {k: inverted_index[k] for k in list(inverted_index)[:5]})\n# print("Sample Title Index:", {k: title_index[k] for k in list(title_index)[:5]})\n# Print the entire inverted index with one term per row\nfor term, postings in list(inverted_index.items())[:10]:\n    print(f"{term}: {postings}")'

 **2.2. Propose test queries:** Define five queries that will be used to evaluate your search engine (e.g., “Indian protest”, “support farmers”)

 **HINT:** How to choose the queries? The selection of the queries is up to you but it’s suggested to select terms based on the popularity (keywords ranked by term frequencies or by TF-IDF, etc...).

In [226]:
test_queries = [
    "protest indian",
    "protest farmer",
    "support farmer",
    "free india ",
    "universitat pompeu fabra" # option that will not appear in the dataset
]

 **2.3. Search function:**

In [227]:
'''# extracted from professors file and changed what was necessary

def search_v0(query, index):
    """
    This function returns the list of documents that contain all the words in the query,
    and the words must appear consecutively within the document.

    Arguments:
    query -- string (query to be searched)
    index -- inverted index created from the documents

    Returns:
    List of document IDs containing all terms in consecutive positions
    """
    query_terms = build_terms(query)  # Preprocess the query terms
    if not query_terms:
        return []  # Return empty if no terms after preprocessing

    # Store documents that match all terms
    matching_docs = set()
    first_term_docs = None  # This will hold the docs for the first term

    for term in query_terms:
        try:
            # Get the postings for the current term
            current_term_postings = index[term]
            current_term_docs = {posting[0] for posting in current_term_postings}

            # Initialize first_term_docs with the current_term_docs for the first term
            if first_term_docs is None:
                first_term_docs = current_term_docs
            else:
                first_term_docs &= current_term_docs  # Keep only common docs

        except KeyError:
            # If the term is not found, we cannot satisfy the query
            return []

    # Now check if any of the common documents contain all terms in consecutive positions
    for doc_id in first_term_docs:
        # Extract positions for all terms in the document
        positions = {term: [] for term in query_terms}  # Store positions for each term
        for term in query_terms:
            for posting in index.get(term, []):
                if posting[0] == doc_id:  # Check if doc_id matches
                    positions[term] = posting[1]  # Get the list of positions

        # Check for consecutive positions across all terms
        if all(positions.values()):  # Check if any term has positions
            # Create a list of all positions from the terms
            all_positions = [pos for term in query_terms for pos in positions[term]]
            all_positions.sort()  # Sort positions

            # Check if there are consecutive positions
            for i in range(len(all_positions) - 1):
                if all_positions[i + 1] - all_positions[i] == 1:
                    matching_docs.add(doc_id)
                    break  # No need to check further for this doc

    return list(matching_docs)
'''

'# extracted from professors file and changed what was necessary\n\ndef search_v0(query, index):\n    """\n    This function returns the list of documents that contain all the words in the query,\n    and the words must appear consecutively within the document.\n\n    Arguments:\n    query -- string (query to be searched)\n    index -- inverted index created from the documents\n\n    Returns:\n    List of document IDs containing all terms in consecutive positions\n    """\n    query_terms = build_terms(query)  # Preprocess the query terms\n    if not query_terms:\n        return []  # Return empty if no terms after preprocessing\n\n    # Store documents that match all terms\n    matching_docs = set()\n    first_term_docs = None  # This will hold the docs for the first term\n\n    for term in query_terms:\n        try:\n            # Get the postings for the current term\n            current_term_postings = index[term]\n            current_term_docs = {posting[0] for posting in current_

In [228]:
'''# search for each query
top = 10  # Num top results to display

# We save the number of results
num_results_search_v0 = []

for query in test_queries:
    ranked_docs = search_v0(query, inverted_index)

    print("\n======================")
    print("Top {} results out of {} for the searched query '{}':\n".format(top, len(ranked_docs), query))
    num_results_search_v0.append([query, len(ranked_docs)])

    for d_id in ranked_docs[:top]:  # limit to top results
        print("page_id= {} - page_title: {}".format(d_id, title_index[d_id]))'''

'# search for each query\ntop = 10  # Num top results to display\n\n# We save the number of results\nnum_results_search_v0 = []\n\nfor query in test_queries:\n    ranked_docs = search_v0(query, inverted_index)\n\n    print("\n======================")\n    print("Top {} results out of {} for the searched query \'{}\':\n".format(top, len(ranked_docs), query))\n    num_results_search_v0.append([query, len(ranked_docs)])\n\n    for d_id in ranked_docs[:top]:  # limit to top results\n        print("page_id= {} - page_title: {}".format(d_id, title_index[d_id]))'

In [229]:
'''def search(query, index):
    """
    This function returns the list of documents that contain all the terms in the query.
    The terms do not need to appear consecutively, only to be present in the document.

    Arguments:
    query -- string (query to be searched)
    index -- inverted index created from the documents

    Returns:
    List of document IDs containing all terms
    """
    query_terms = build_terms(query)  # Preprocess the query terms
    if not query_terms:
        return []  # Return empty if no terms after preprocessing

    # Initialize the intersection of documents
    matching_docs = None

    for term in query_terms:
        try:
            # Get the documents that contain the current term
            current_term_docs = {posting[0] for posting in index[term]}

            # Initialize with documents of the first term
            if matching_docs is None:
                matching_docs = current_term_docs
            else:
                # Perform intersection to keep only common documents
                matching_docs &= current_term_docs

        except KeyError:
            # If any term is not found in the index, the query cannot be satisfied
            return []

    # Convert the resulting set to a list
    return list(matching_docs) if matching_docs else []
'''

'def search(query, index):\n    """\n    This function returns the list of documents that contain all the terms in the query.\n    The terms do not need to appear consecutively, only to be present in the document.\n\n    Arguments:\n    query -- string (query to be searched)\n    index -- inverted index created from the documents\n\n    Returns:\n    List of document IDs containing all terms\n    """\n    query_terms = build_terms(query)  # Preprocess the query terms\n    if not query_terms:\n        return []  # Return empty if no terms after preprocessing\n\n    # Initialize the intersection of documents\n    matching_docs = None\n\n    for term in query_terms:\n        try:\n            # Get the documents that contain the current term\n            current_term_docs = {posting[0] for posting in index[term]}\n\n            # Initialize with documents of the first term\n            if matching_docs is None:\n                matching_docs = current_term_docs\n            else:\n       

In [230]:
'''# search for each query
top = 10  # Num top results to display

# We save the number of results
num_results_search = []

for query in test_queries:
    ranked_docs = search(query, inverted_index)

    print("\n======================")
    print("Top {} results out of {} for the searched query '{}':\n".format(top, len(ranked_docs), query))
    num_results_search.append([query, len(ranked_docs)])

    for d_id in ranked_docs[:top]:  # limit to top results
        print("page_id= {} - page_title: {}".format(d_id, title_index[d_id]))'''

'# search for each query\ntop = 10  # Num top results to display\n\n# We save the number of results\nnum_results_search = []\n\nfor query in test_queries:\n    ranked_docs = search(query, inverted_index)\n\n    print("\n======================")\n    print("Top {} results out of {} for the searched query \'{}\':\n".format(top, len(ranked_docs), query))\n    num_results_search.append([query, len(ranked_docs)])\n\n    for d_id in ranked_docs[:top]:  # limit to top results\n        print("page_id= {} - page_title: {}".format(d_id, title_index[d_id]))'

In [231]:
'''# Assuming num_results_serach and num_results_serach_v0 are of equal length and contain pairs of [query, num_results]
print("Comparison of results between search versions:\n")

for (query_1, results_1), (query_2, results_2) in zip(num_results_search, num_results_search_v0):
    # Ensure the queries match
    if query_1 == query_2:
        print("Query: '{}'".format(query_1))
        print(" Version 0 results: {} | Final version results: {} | Difference: {}".format(results_2, results_1, results_1 - results_2))
        print("-------------------------")
    else:
        print("Warning: Queries do not match between versions.")'''


'# Assuming num_results_serach and num_results_serach_v0 are of equal length and contain pairs of [query, num_results]\nprint("Comparison of results between search versions:\n")\n\nfor (query_1, results_1), (query_2, results_2) in zip(num_results_search, num_results_search_v0):\n    # Ensure the queries match\n    if query_1 == query_2:\n        print("Query: \'{}\'".format(query_1))\n        print(" Version 0 results: {} | Final version results: {} | Difference: {}".format(results_2, results_1, results_1 - results_2))\n        print("-------------------------")\n    else:\n        print("Warning: Queries do not match between versions.")'

**2.4. Rankyour results:** Implement the TF-IDF algorithm and provide ranking based results.

In [232]:
'''# extracted from professors file and changed what was necessary

def create_index_tfidf(lines, num_documents):
    """
    Optimized version of inverted index and tf-idf computation.
    """
    index = defaultdict(list)
    tf = defaultdict(list)  # term frequencies
    df = defaultdict(int)  # document frequencies
    title_index = {}
    term_count_per_doc = defaultdict(int)  # To store term count per doc for normalization

    # First pass: create index and compute term frequencies (TF) and document frequencies (DF)
    for doc_id, tweet_text in lines:
        title_index[doc_id] = tweet_text
        terms = build_terms(tweet_text)

        current_page_index = defaultdict(list)
        term_frequencies = defaultdict(int)  # Frequency of each term in the current document

        # Count term frequencies and track positions
        for position, term in enumerate(terms):
            current_page_index[term].append(position)
            term_frequencies[term] += 1

        # Update global index and DF
        for term, positions in current_page_index.items():
            index[term].append([doc_id, positions])
            df[term] += 1  # Document frequency
            term_count_per_doc[doc_id] += len(positions)

        # Normalize term frequencies for TF
        norm = math.sqrt(sum(f**2 for f in term_frequencies.values()))
        for term, freq in term_frequencies.items():
            tf[term].append(np.round(freq / norm, 4))

    # Second pass: compute IDF
    idf = {term: np.round(math.log(num_documents / df[term]), 4) for term in df}

    return index, tf, df, idf, title_index'''

'# extracted from professors file and changed what was necessary\n\ndef create_index_tfidf(lines, num_documents):\n    """\n    Optimized version of inverted index and tf-idf computation.\n    """\n    index = defaultdict(list)\n    tf = defaultdict(list)  # term frequencies\n    df = defaultdict(int)  # document frequencies\n    title_index = {}\n    term_count_per_doc = defaultdict(int)  # To store term count per doc for normalization\n\n    # First pass: create index and compute term frequencies (TF) and document frequencies (DF)\n    for doc_id, tweet_text in lines:\n        title_index[doc_id] = tweet_text\n        terms = build_terms(tweet_text)\n\n        current_page_index = defaultdict(list)\n        term_frequencies = defaultdict(int)  # Frequency of each term in the current document\n\n        # Count term frequencies and track positions\n        for position, term in enumerate(terms):\n            current_page_index[term].append(position)\n            term_frequencies[term]

In [233]:
'''start_time = time.time()
num_documents = len(lines)
index, tf, df, idf, title_index = create_index_tfidf(lines, num_documents)
print("Total time to create the index: {} seconds" .format(np.round(time.time() - start_time, 2)))'''

'start_time = time.time()\nnum_documents = len(lines)\nindex, tf, df, idf, title_index = create_index_tfidf(lines, num_documents)\nprint("Total time to create the index: {} seconds" .format(np.round(time.time() - start_time, 2)))'

In [234]:
'''# extracted from professors file and changed what was necessary

def rank_documents(terms, docs, index, idf, tf, title_index):
    """
    Perform the ranking of the results of a search based on the tf-idf weights

    Argument:
    terms -- list of query terms
    docs -- list of documents, to rank, matching the query
    index -- inverted index data structure
    idf -- inverted document frequencies
    tf -- term frequencies
    title_index -- mapping between page id and page title

    Returns:
    Print the list of ranked documents
    """

    # I'm interested only on the element of the docVector corresponding to the query terms
    # The remaining elements would became 0 when multiplied to the query_vector
    doc_vectors = defaultdict(lambda: [0] * len(terms)) # I call doc_vectors[k] for a nonexistent key k, the key-value pair (k,[0]*len(terms)) will be automatically added to the dictionary
    query_vector = [0] * len(terms)

    # compute the norm for the query tf
    query_terms_count = collections.Counter(terms)  # get the frequency of each term in the query.
    # Example: collections.Counter(["hello","hello","world"]) --> Counter({'hello': 2, 'world': 1})
    # HINT: use when computing tf for query_vector

    query_norm = la.norm(list(query_terms_count.values()))

    for termIndex, term in enumerate(terms):  #termIndex is the index of the term in the query
        if term not in index:
            continue

        ## Compute tf*idf(normalize TF as done with documents)
        query_vector[termIndex] = (query_terms_count[term] / query_norm) * idf[term]

        # Generate doc_vectors for matching docs
        for doc_index, (doc, postings) in enumerate(index[term]):
            # Example of [doc_index, (doc, postings)]
            # 0 (26, array('I', [1, 4, 12, 15, 22, 28, 32, 43, 51, 68, 333, 337]))
            # 1 (33, array('I', [26, 33, 57, 71, 87, 104, 109]))
            # term is in doc 26 in positions 1,4, .....
            # term is in doc 33 in positions 26,33, .....

            #tf[term][0] will contain the tf of the term "term" in the doc 26
            if doc in docs:
                doc_vectors[doc][termIndex] = tf[term][doc_index] * idf[term]  # TODO: check if multiply for idf

    # Calculate the score of each doc
    # compute the cosine similarity between queyVector and each docVector:
    # HINT: you can use the dot product because in case of normalized vectors it corresponds to the cosine similarity
    # see np.dot

    doc_scores = [[np.dot(curDocVec, query_vector), doc] for doc, curDocVec in doc_vectors.items()]
    doc_scores.sort(reverse=True)
    result_docs = [x[1] for x in doc_scores]
    #print document titles instead if document id's
    #result_docs=[ title_index[x] for x in result_docs ]

    return result_docs'''

'# extracted from professors file and changed what was necessary\n\ndef rank_documents(terms, docs, index, idf, tf, title_index):\n    """\n    Perform the ranking of the results of a search based on the tf-idf weights\n\n    Argument:\n    terms -- list of query terms\n    docs -- list of documents, to rank, matching the query\n    index -- inverted index data structure\n    idf -- inverted document frequencies\n    tf -- term frequencies\n    title_index -- mapping between page id and page title\n\n    Returns:\n    Print the list of ranked documents\n    """\n\n    # I\'m interested only on the element of the docVector corresponding to the query terms\n    # The remaining elements would became 0 when multiplied to the query_vector\n    doc_vectors = defaultdict(lambda: [0] * len(terms)) # I call doc_vectors[k] for a nonexistent key k, the key-value pair (k,[0]*len(terms)) will be automatically added to the dictionary\n    query_vector = [0] * len(terms)\n\n    # compute the norm for

In [235]:
'''# extracted from professors file and changed what was necessary

def search_tf_idf_v0(query, index):
    """
    This function returns the list of documents that contain all the words in the query,
    and the words must appear consecutively within the document.

    Arguments:
    query -- string (query to be searched)
    index -- inverted index created from the documents

    Returns:
    List of document IDs containing all terms in consecutive positions
    """
    query_terms = build_terms(query)  # Preprocess the query terms
    if not query_terms:
        return []  # Return empty if no terms after preprocessing

    # Store documents that match all terms
    matching_docs = set()
    first_term_docs = None  # This will hold the docs for the first term

    for term in query_terms:
        try:
            # Get the postings for the current term
            current_term_postings = index[term]
            current_term_docs = {posting[0] for posting in current_term_postings}
            #matching_docs = matching_docs.union(current_term_docs)

            # Initialize first_term_docs with the current_term_docs for the first term
            if first_term_docs is None:
                first_term_docs = current_term_docs
            else:
                first_term_docs &= current_term_docs  # Keep only common docs

        except:
            # If the term is not found, we cannot satisfy the query
            pass

    # Now check if any of the common documents contain all terms in consecutive positions
    for doc_id in first_term_docs:
        # Extract positions for all terms in the document
        positions = {term: [] for term in query_terms}  # Store positions for each term
        for term in query_terms:
            for posting in index.get(term, []):
                if posting[0] == doc_id:  # Check if doc_id matches
                    positions[term] = posting[1]  # Get the list of positions

        # Check for consecutive positions across all terms
        if all(positions.values()):  # Check if any term has positions
            # Create a list of all positions from the terms
            all_positions = [pos for term in query_terms for pos in positions[term]]
            all_positions.sort()  # Sort positions

            # Check if there are consecutive positions
            for i in range(len(all_positions) - 1):
                if all_positions[i + 1] - all_positions[i] == 1:
                    matching_docs.add(doc_id)
                    break  # No need to check further for this doc

    matching_docs = list(matching_docs)
    ranked_docs = rank_documents(query, matching_docs, index, idf, tf, title_index)
    #print(ranked_docs)
    return ranked_docs'''


'# extracted from professors file and changed what was necessary\n\ndef search_tf_idf_v0(query, index):\n    """\n    This function returns the list of documents that contain all the words in the query,\n    and the words must appear consecutively within the document.\n\n    Arguments:\n    query -- string (query to be searched)\n    index -- inverted index created from the documents\n\n    Returns:\n    List of document IDs containing all terms in consecutive positions\n    """\n    query_terms = build_terms(query)  # Preprocess the query terms\n    if not query_terms:\n        return []  # Return empty if no terms after preprocessing\n\n    # Store documents that match all terms\n    matching_docs = set()\n    first_term_docs = None  # This will hold the docs for the first term\n\n    for term in query_terms:\n        try:\n            # Get the postings for the current term\n            current_term_postings = index[term]\n            current_term_docs = {posting[0] for posting in c

In [236]:
'''def search_tf_idf(query, index):
    """
    This function returns the list of documents that contain all terms in the query,
    without requiring them to appear in consecutive order. It then ranks the documents
    by their TF-IDF scores.

    Arguments:
    query -- string (query to be searched)
    index -- inverted index created from the documents

    Returns:
    Ranked list of document IDs based on TF-IDF scores
    """
    query_terms = build_terms(query)  # Preprocess the query terms
    if not query_terms:
        return []  # Return empty if no terms after preprocessing

    # Store documents that match all terms
    matching_docs = None

    for term in query_terms:
        try:
            # Get the documents that contain the current term
            current_term_docs = {posting[0] for posting in index[term]}

            # Initialize matching_docs with documents of the first term
            if matching_docs is None:
                matching_docs = current_term_docs
            else:
                # Perform intersection to keep only common documents
                matching_docs &= current_term_docs

        except KeyError:
            # If any term is not found in the index, the query cannot be satisfied
            return []

    # Rank the matching documents based on TF-IDF
    ranked_docs = rank_documents(query, list(matching_docs), index, idf, tf, title_index)
    return ranked_docs'''


'def search_tf_idf(query, index):\n    """\n    This function returns the list of documents that contain all terms in the query,\n    without requiring them to appear in consecutive order. It then ranks the documents\n    by their TF-IDF scores.\n\n    Arguments:\n    query -- string (query to be searched)\n    index -- inverted index created from the documents\n\n    Returns:\n    Ranked list of document IDs based on TF-IDF scores\n    """\n    query_terms = build_terms(query)  # Preprocess the query terms\n    if not query_terms:\n        return []  # Return empty if no terms after preprocessing\n\n    # Store documents that match all terms\n    matching_docs = None\n\n    for term in query_terms:\n        try:\n            # Get the documents that contain the current term\n            current_term_docs = {posting[0] for posting in index[term]}\n\n            # Initialize matching_docs with documents of the first term\n            if matching_docs is None:\n                matching_d

In [237]:
'''# Execute search for each query
top = 10  # Number of top results to display

num_results_search_tf_idf = []

for query in test_queries:
    ranked_docs = search_tf_idf(query, index)  # Call the search function with the query

    print("\n======================")
    print("Top {} results out of {} for the searched query '{}':\n".format(top, len(ranked_docs), query))
    num_results_search_tf_idf.append([query, len(ranked_docs)])

    for d_id in ranked_docs[:top]:  # Limit to top results
        print("page_id= {} - page_title: {}".format(d_id, title_index[d_id]))'''

'# Execute search for each query\ntop = 10  # Number of top results to display\n\nnum_results_search_tf_idf = []\n\nfor query in test_queries:\n    ranked_docs = search_tf_idf(query, index)  # Call the search function with the query\n\n    print("\n======================")\n    print("Top {} results out of {} for the searched query \'{}\':\n".format(top, len(ranked_docs), query))\n    num_results_search_tf_idf.append([query, len(ranked_docs)])\n\n    for d_id in ranked_docs[:top]:  # Limit to top results\n        print("page_id= {} - page_title: {}".format(d_id, title_index[d_id]))'

In [238]:
'''# Assuming num_results_search_tf_idf and num_results_search are of equal length and contain pairs of [query, num_results]
print("Comparison of results between search versions:\n")

for (query_1, results_tf_idf), (query_2, results_normal) in zip(num_results_search_tf_idf, num_results_search):
    # Ensure the queries match
    if query_1 == query_2:
        print("Query: '{}'".format(query_1))
        print("TF-IDF Version results: {} | Normal Version results: {} | Difference: {}".format(results_tf_idf, results_normal, results_normal - results_tf_idf))
        print("-------------------------")
    else:
        print("Warning: Queries do not match between versions.")
'''

'# Assuming num_results_search_tf_idf and num_results_search are of equal length and contain pairs of [query, num_results]\nprint("Comparison of results between search versions:\n")\n\nfor (query_1, results_tf_idf), (query_2, results_normal) in zip(num_results_search_tf_idf, num_results_search):\n    # Ensure the queries match\n    if query_1 == query_2:\n        print("Query: \'{}\'".format(query_1))\n        print("TF-IDF Version results: {} | Normal Version results: {} | Difference: {}".format(results_tf_idf, results_normal, results_normal - results_tf_idf))\n        print("-------------------------")\n    else:\n        print("Warning: Queries do not match between versions.")\n'

<font size="+2">3. Evaluation</font>

3.1.A baseline with 2 queries and the ground truth files for each query will be given to you, using a subset of documents from the dataset.

a. Query1: people's rights?

b. Query2: Indian government?

In [239]:
queries = ["people's rights?", "Indian government?"]


To compute Precision@K, Mean Average Precision and Mean Reciprocal Rank, we need binary relevance (1 = relevant, 0 = not relevant).

For the two provided queries, we will use the dataset evaulation_gt.csv that contains the colum label (with values 0 and 1) stating the relevance of the ground truth.

In [240]:
'''# for the two queries provided. using evaluation_gt.csv
search_results_q = pd.read_csv(evaluation_path, delimiter=';')
search_results_q.head()'''

"# for the two queries provided. using evaluation_gt.csv\nsearch_results_q = pd.read_csv(evaluation_path, delimiter=';')\nsearch_results_q.head()"

For those two queries, we must compute their corresponding predicte_relevance value

In [241]:
'''def set_predicted_relevance(dataframe, queries):
    dataframe['predicted_relevance'] = -5

    # Perform TF-IDF search for each query and assign predicted relevance
    for query_id, query in enumerate(queries, start=1):
        print(f"Processing Query {query_id}: {query}")

        # Perform the search and ranking. we decided to do it with the search() function instead of the search_tf_idf() function
        ranked_docs = search_tf_idf(query, index)

        # Assign predicted relevance score (e.g., rank position as relevance) for the top-ranked documents
        num_docs = len(ranked_docs)
        max_score = 4  # Maximum relevance score
        min_score = -5  # Minimum relevance score

        # linear interpolation
        scores = np.linspace(max_score, min_score, num_docs)
        for rank, doc_id in enumerate(ranked_docs, start=1):
            # Update the predicted relevance for the matching query_id and docId
            search_results_q.loc[(dataframe['query_id'] == query_id) & (search_results_q['docId'] == doc_id), 'predicted_relevance'] = scores[rank-1]  # relevance score

set_predicted_relevance(search_results_q, queries)'''

'def set_predicted_relevance(dataframe, queries):\n    dataframe[\'predicted_relevance\'] = -5\n\n    # Perform TF-IDF search for each query and assign predicted relevance\n    for query_id, query in enumerate(queries, start=1):\n        print(f"Processing Query {query_id}: {query}")\n\n        # Perform the search and ranking. we decided to do it with the search() function instead of the search_tf_idf() function\n        ranked_docs = search_tf_idf(query, index)\n\n        # Assign predicted relevance score (e.g., rank position as relevance) for the top-ranked documents\n        num_docs = len(ranked_docs)\n        max_score = 4  # Maximum relevance score\n        min_score = -5  # Minimum relevance score\n\n        # linear interpolation\n        scores = np.linspace(max_score, min_score, num_docs)\n        for rank, doc_id in enumerate(ranked_docs, start=1):\n            # Update the predicted relevance for the matching query_id and docId\n            search_results_q.loc[(dataframe

In [242]:
'''display(search_results_q.head())'''

'display(search_results_q.head())'

*You will be the expert judges, so you will be setting the ground truth for each document and query in a binary way for the test queries that you defined in step 2 at the indexing stage.*


For our own queries, we must create a ground truth. We will follow again the same metrics as the evaluation_gt.csv file, setting 5 levels of relevance (from 0 to 5) and setting them randomly. Finally, we will consider as relevant all documents that have actual score equal or higher than  2 , and not-relevant the remaining documents.

In [243]:
'''search_results_tq = []  # results for the search on test_queries

# execute search for each query
# loop through each query in the test_queries

random.seed(42)

for i, query in enumerate(test_queries, start=1):

    # Perform the search and ranking. we decided to do it with the search() function instead of the search_tf_idf() function
    ranked_docs = search_tf_idf(query, index)

    num_docs = len(ranked_docs)
    max_score = 4  # Maximum relevance score
    min_score = -5  # Minimum relevance score

    # linear interpolation
    scores = np.linspace(max_score, min_score, num_docs)

    # Assign predicted relevance score (e.g., rank position as relevance) for the top-ranked documents
    for rank, doc_id in enumerate(ranked_docs[:top], start=0):  # Limit to top results

        # set a random value for doc_score for each document since it is not provided in the dataset
        doc_score = random.choice([0.0, 1.0, 2.0, 3.0, 4.0])

        # Update the search_results_2 list with the predicted relevance score
        score = scores[rank]
        search_results_tq.append({
            'docId': doc_id,
            'query_id': i,
            'predicted_relevance': score,
            'doc_score':doc_score
        })

search_results_tq = pd.DataFrame(search_results_tq)'''

"search_results_tq = []  # results for the search on test_queries\n\n# execute search for each query\n# loop through each query in the test_queries\n\nrandom.seed(42)\n\nfor i, query in enumerate(test_queries, start=1):\n\n    # Perform the search and ranking. we decided to do it with the search() function instead of the search_tf_idf() function\n    ranked_docs = search_tf_idf(query, index)\n\n    num_docs = len(ranked_docs)\n    max_score = 4  # Maximum relevance score\n    min_score = -5  # Minimum relevance score\n\n    # linear interpolation\n    scores = np.linspace(max_score, min_score, num_docs)\n\n    # Assign predicted relevance score (e.g., rank position as relevance) for the top-ranked documents\n    for rank, doc_id in enumerate(ranked_docs[:top], start=0):  # Limit to top results\n\n        # set a random value for doc_score for each document since it is not provided in the dataset\n        doc_score = random.choice([0.0, 1.0, 2.0, 3.0, 4.0])\n\n        # Update the searc

In [244]:
'''display(search_results_tq.head())'''

'display(search_results_tq.head())'

In [245]:
'''# since we didn't have the doc_score value for our queris, we setted randomly
# and followed the same pattern as professors did in the lab preparation, which is the following

# set label values of relevance. If doc_score >2 then label=1 otherwise 0
search_results_tq["label"] = search_results_tq["doc_score"].apply(lambda y: 1 if y >= 2 else 0)
search_results_tq.head()'''

'# since we didn\'t have the doc_score value for our queris, we setted randomly\n# and followed the same pattern as professors did in the lab preparation, which is the following\n\n# set label values of relevance. If doc_score >2 then label=1 otherwise 0\nsearch_results_tq["label"] = search_results_tq["doc_score"].apply(lambda y: 1 if y >= 2 else 0)\nsearch_results_tq.head()'

In [246]:
'''# for query 1
result_query_id_1 = search_results_tq[search_results_tq['query_id'] == 1].head(10)
#result_query_id_1 = search_results_tq[(search_results_tq['label'] == 0) & (search_results_tq['query_id'] == 1)].head(20)
display(result_query_id_1)

# for query 2
result_query_id_2 = search_results_tq[search_results_tq['query_id'] == 2].head(10)
#result_query_id_2 = search_results_tq[(search_results_tq['label'] == 1) & (search_results_tq['query_id'] == 2)].head(20)
display(result_query_id_2)
'''

"# for query 1\nresult_query_id_1 = search_results_tq[search_results_tq['query_id'] == 1].head(10)\n#result_query_id_1 = search_results_tq[(search_results_tq['label'] == 0) & (search_results_tq['query_id'] == 1)].head(20)\ndisplay(result_query_id_1)\n\n# for query 2\nresult_query_id_2 = search_results_tq[search_results_tq['query_id'] == 2].head(10)\n#result_query_id_2 = search_results_tq[(search_results_tq['label'] == 1) & (search_results_tq['query_id'] == 2)].head(20)\ndisplay(result_query_id_2)\n"

*3.2. For the prior evaluation components you must evaluate your algorithm by using different evaluation techniques:*


* *Precision @K(P@K)*
* *Recall @K(R@K)*
* *Average Precision @K(P@K)*
* *F1-Score @K*
* *Mean Average Precision(MAP)*
* *Mean Reciprocal Rank(MRR)*
* *Normalized Discounted Cumulative Gain(NDCG)*

*Only for the second component(your queries) comment on each of the evaluation techniques stating how they differ,and which information gives each of them. Analyze your results.*


**3.2.1 Precision @K(P@K)**

In [247]:
'''# extracted from professor's practise lab file

def precision_at_k(y_true, y_score, k=10):
    ''
    Parameters
    ----------
    y_true: Ground truth (true relevance labels).
    y_score: Predicted scores.
    k : number of documents to consider.

    Returns
    -------
    precision @k : float
    ''
    # Sort both y_true and y_score by the scores in descending order
    sorted_indices = np.argsort(y_score)[::-1]
    y_true_sorted = np.array(y_true)[sorted_indices][:k]

    # Calculate the number of relevant documents in the top-k results
    relevant = np.sum(y_true_sorted)

    # Return precision@k
    return relevant / k
'''

"# extracted from professor's practise lab file\n\ndef precision_at_k(y_true, y_score, k=10):\n    ''\n    Parameters\n    ----------\n    y_true: Ground truth (true relevance labels).\n    y_score: Predicted scores.\n    k : number of documents to consider.\n\n    Returns\n    -------\n    precision @k : float\n    ''\n    # Sort both y_true and y_score by the scores in descending order\n    sorted_indices = np.argsort(y_score)[::-1]\n    y_true_sorted = np.array(y_true)[sorted_indices][:k]\n\n    # Calculate the number of relevant documents in the top-k results\n    relevant = np.sum(y_true_sorted)\n\n    # Return precision@k\n    return relevant / k\n"

In [248]:
'''def precision_at_k_v2(y_true, y_score, k=10):
    ''
    Parameters
    ----------
    y_true : List of tuples representing ground truth (true relevance labels).
             Each tuple is (doc_id, label), where label is 1 for relevant, 0 otherwise.
    y_score : List of document IDs retrieved, ranked by relevance.
    k : int, number of documents to consider (top-k).

    Returns
    -------
    precision_at_k : float
    ''

    # Get the top k documents from y_score
    y_score_k = y_score[:k]  # Take only the first k elements

    # Initialize a counter for relevant documents
    relevant_count = 0

    # Check each document in y_true against the top k documents in y_score
    for doc_id_true, label in y_true:
        if label == 1 and doc_id_true in y_score_k:
            relevant_count += 1

    # Calculate precision@k
    precision_at_k = relevant_count / k if k > 0 else 0  # Prevent division by zero

    return precision_at_k
'''

"def precision_at_k_v2(y_true, y_score, k=10):\n    ''\n    Parameters\n    ----------\n    y_true : List of tuples representing ground truth (true relevance labels).\n             Each tuple is (doc_id, label), where label is 1 for relevant, 0 otherwise.\n    y_score : List of document IDs retrieved, ranked by relevance.\n    k : int, number of documents to consider (top-k).\n\n    Returns\n    -------\n    precision_at_k : float\n    ''\n\n    # Get the top k documents from y_score\n    y_score_k = y_score[:k]  # Take only the first k elements\n\n    # Initialize a counter for relevant documents\n    relevant_count = 0\n\n    # Check each document in y_true against the top k documents in y_score\n    for doc_id_true, label in y_true:\n        if label == 1 and doc_id_true in y_score_k:\n            relevant_count += 1\n\n    # Calculate precision@k\n    precision_at_k = relevant_count / k if k > 0 else 0  # Prevent division by zero\n\n    return precision_at_k\n"

Test for provided queries:

In [249]:
'''# NOT THE FINAL VERSION (explained in the report)
precision_q2 = {}
k = 10  # Set the value of k

for query_id in search_results_q['query_id'].unique():  # Loop through unique query_ids
    if pd.isna(query_id):  # Check if query_id is NaN
        continue  # Skip to the next iteration if query_id is NaN

    # Retrieve y_true for the current query_id
    current_query_results = search_results_q[search_results_q['query_id'] == query_id]
    y_true = current_query_results[['docId', 'label']].values.tolist()  # Get docID and label as tuples

    # Execute search_tf_idf to get y_score
    search_results = search(queries[int(query_id)-1], index)  # Assuming query_id is the search term here
    y_score = search_results  # Get list of document IDs from the search results

    # Calculate precision using precision_at_k_v2
    precision_q2[query_id] = precision_at_k_v2(y_true, y_score, k)
    print("==> Precision@{}: {}\n".format(k, precision_q2[query_id]))'''

'# NOT THE FINAL VERSION (explained in the report)\nprecision_q2 = {}\nk = 10  # Set the value of k\n\nfor query_id in search_results_q[\'query_id\'].unique():  # Loop through unique query_ids\n    if pd.isna(query_id):  # Check if query_id is NaN\n        continue  # Skip to the next iteration if query_id is NaN\n\n    # Retrieve y_true for the current query_id\n    current_query_results = search_results_q[search_results_q[\'query_id\'] == query_id]\n    y_true = current_query_results[[\'docId\', \'label\']].values.tolist()  # Get docID and label as tuples\n\n    # Execute search_tf_idf to get y_score\n    search_results = search(queries[int(query_id)-1], index)  # Assuming query_id is the search term here\n    y_score = search_results  # Get list of document IDs from the search results\n\n    # Calculate precision using precision_at_k_v2\n    precision_q2[query_id] = precision_at_k_v2(y_true, y_score, k)\n    print("==> Precision@{}: {}\n".format(k, precision_q2[query_id]))'

In [250]:
'''precision_q = {}
for query_id, query in enumerate(queries, start=1):

    current_query = query_id
    current_query_res = search_results_q[search_results_q["query_id"] == current_query]

    k = 10
    precision_q[query_id] = precision_at_k(current_query_res["label"], current_query_res["predicted_relevance"], k)
    print("==> Precision@{}: {}\n".format(k,precision_q[query_id]))

    print("\nCheck on the dataset sorted by score:\n")
    #current_query_res.sort_values("score", ascending=False).head(k)
    display(current_query_res.sort_values("predicted_relevance", ascending=False).head(k))
    print()'''

'precision_q = {}\nfor query_id, query in enumerate(queries, start=1):\n\n    current_query = query_id\n    current_query_res = search_results_q[search_results_q["query_id"] == current_query]\n\n    k = 10\n    precision_q[query_id] = precision_at_k(current_query_res["label"], current_query_res["predicted_relevance"], k)\n    print("==> Precision@{}: {}\n".format(k,precision_q[query_id]))\n\n    print("\nCheck on the dataset sorted by score:\n")\n    #current_query_res.sort_values("score", ascending=False).head(k)\n    display(current_query_res.sort_values("predicted_relevance", ascending=False).head(k))\n    print()'

Test for created test_queries:

In [251]:
'''precision_tq = {}
for query_id, query in enumerate(test_queries, start=1):

    current_query = query_id
    current_query_res = search_results_tq[search_results_tq["query_id"] == current_query]

    k = 10
    precision_tq[query_id] = precision_at_k(current_query_res["label"], current_query_res["predicted_relevance"], k)
    print("==> Precision@{}: {}\n".format(k,precision_tq[query_id]))

    print("\nCheck on the dataset sorted by score:\n")
    #current_query_res.sort_values("score", ascending=False).head(k)
    display(current_query_res.sort_values("predicted_relevance", ascending=False).head(k))
    print()'''

'precision_tq = {}\nfor query_id, query in enumerate(test_queries, start=1):\n\n    current_query = query_id\n    current_query_res = search_results_tq[search_results_tq["query_id"] == current_query]\n\n    k = 10\n    precision_tq[query_id] = precision_at_k(current_query_res["label"], current_query_res["predicted_relevance"], k)\n    print("==> Precision@{}: {}\n".format(k,precision_tq[query_id]))\n\n    print("\nCheck on the dataset sorted by score:\n")\n    #current_query_res.sort_values("score", ascending=False).head(k)\n    display(current_query_res.sort_values("predicted_relevance", ascending=False).head(k))\n    print()'

**3.2.2 Recall @K(R@K)**


In [252]:
'''def recall_at_k(y_true, y_pred, k=10):
    """
    Calculate the recall at k.

    Parameters:
    - y_true (array-like): True binary labels (1 for relevant, 0 for irrelevant).
    - y_pred (array-like): Predicted scores for the items.
    - k (int): Number of top-ranked items to consider.

    Returns:
    - recall (float): Recall score at k.
    """
    # Get the top k indices from y_pred
    top_k_indices = np.argsort(y_pred)[::-1][:k]
    # Filter y_true to only include the top k items
    y_true_at_k = np.array(y_true)[top_k_indices]
    # Calculate the number of relevant items in the top k
    relevant_at_k = np.sum(y_true_at_k)
    # Calculate the total number of relevant items
    total_relevant = np.sum(y_true)
    # Return recall score at k
    recall_at_k = relevant_at_k / total_relevant if total_relevant != 0 else 0.0
    return recall_at_k'''

'def recall_at_k(y_true, y_pred, k=10):\n    """\n    Calculate the recall at k.\n\n    Parameters:\n    - y_true (array-like): True binary labels (1 for relevant, 0 for irrelevant).\n    - y_pred (array-like): Predicted scores for the items.\n    - k (int): Number of top-ranked items to consider.\n\n    Returns:\n    - recall (float): Recall score at k.\n    """\n    # Get the top k indices from y_pred\n    top_k_indices = np.argsort(y_pred)[::-1][:k]\n    # Filter y_true to only include the top k items\n    y_true_at_k = np.array(y_true)[top_k_indices]\n    # Calculate the number of relevant items in the top k\n    relevant_at_k = np.sum(y_true_at_k)\n    # Calculate the total number of relevant items\n    total_relevant = np.sum(y_true)\n    # Return recall score at k\n    recall_at_k = relevant_at_k / total_relevant if total_relevant != 0 else 0.0\n    return recall_at_k'

In [253]:
'''recall_q = {}

for query_id, query in enumerate(queries, start=1):

    current_query = query_id
    current_query_res = search_results_q[search_results_q["query_id"] == current_query]

    recall_q[query_id] = recall_at_k(np.array(current_query_res["label"]), np.array(current_query_res["predicted_relevance"]), k)
    print(query_id,":",recall_q[query_id])'''

'recall_q = {}\n\nfor query_id, query in enumerate(queries, start=1):\n\n    current_query = query_id\n    current_query_res = search_results_q[search_results_q["query_id"] == current_query]\n\n    recall_q[query_id] = recall_at_k(np.array(current_query_res["label"]), np.array(current_query_res["predicted_relevance"]), k)\n    print(query_id,":",recall_q[query_id])'

In [254]:
'''recall_tq = {}

for query_id, query in enumerate(test_queries, start=1):

    current_query = query_id
    current_query_res = search_results_tq[search_results_tq["query_id"] == current_query]

    recall_tq[query_id] = recall_at_k(np.array(current_query_res["label"]), np.array(current_query_res["predicted_relevance"]), k)
    print(query_id,":",recall_tq[query_id])'''

'recall_tq = {}\n\nfor query_id, query in enumerate(test_queries, start=1):\n\n    current_query = query_id\n    current_query_res = search_results_tq[search_results_tq["query_id"] == current_query]\n\n    recall_tq[query_id] = recall_at_k(np.array(current_query_res["label"]), np.array(current_query_res["predicted_relevance"]), k)\n    print(query_id,":",recall_tq[query_id])'

**3.2.3 Average Precision @K(P@K)**


In [255]:
'''# extracted from professor's practise lab file

def avg_precision_at_k(y_true, y_score, k=10):
    """
    Parameters
    ----------
    y_true: Ground truth (true relevance labels).
    y_score: Predicted scores.
    k : number of doc to consider.

    Returns
    -------
    average precision @k : float
    """
    # get the list of indexes of the predicted score sorted in descending order.
    order = np.argsort(y_score)[::-1]

    prec_at_i = 0
    prec_at_i_list = []
    number_of_relevant = 0
    number_to_iterate = min(k, len(order))

    for i in range(number_to_iterate):
        if y_true[order[i]] == 1:
            number_of_relevant += 1
            prec_at_i = number_of_relevant / (i + 1)
            prec_at_i_list.append(prec_at_i)

    if number_of_relevant == 0:
        return 0
    else:
      return np.sum(prec_at_i_list) / number_of_relevant'''

'# extracted from professor\'s practise lab file\n\ndef avg_precision_at_k(y_true, y_score, k=10):\n    """\n    Parameters\n    ----------\n    y_true: Ground truth (true relevance labels).\n    y_score: Predicted scores.\n    k : number of doc to consider.\n\n    Returns\n    -------\n    average precision @k : float\n    """\n    # get the list of indexes of the predicted score sorted in descending order.\n    order = np.argsort(y_score)[::-1]\n\n    prec_at_i = 0\n    prec_at_i_list = []\n    number_of_relevant = 0\n    number_to_iterate = min(k, len(order))\n\n    for i in range(number_to_iterate):\n        if y_true[order[i]] == 1:\n            number_of_relevant += 1\n            prec_at_i = number_of_relevant / (i + 1)\n            prec_at_i_list.append(prec_at_i)\n\n    if number_of_relevant == 0:\n        return 0\n    else:\n      return np.sum(prec_at_i_list) / number_of_relevant'

Test for provided queries:

In [256]:
'''avg_precision_q = {}
for query_id, query in enumerate(queries, start=1):

    current_query = query_id
    current_query_res = search_results_q[search_results_q["query_id"] == current_query]

    avg_precision_q[query_id] = avg_precision_at_k(np.array(current_query_res["label"]), np.array(current_query_res["predicted_relevance"]), k)
    print(query_id,":",avg_precision_q[query_id])
'''

'avg_precision_q = {}\nfor query_id, query in enumerate(queries, start=1):\n\n    current_query = query_id\n    current_query_res = search_results_q[search_results_q["query_id"] == current_query]\n\n    avg_precision_q[query_id] = avg_precision_at_k(np.array(current_query_res["label"]), np.array(current_query_res["predicted_relevance"]), k)\n    print(query_id,":",avg_precision_q[query_id])\n'

Test for created test_queries:

In [257]:
'''avg_precision_tq = {}
for query_id, query in enumerate(test_queries, start=1):

    current_query = query_id
    current_query_res = search_results_tq[search_results_tq["query_id"] == current_query]
    avg_precision_tq[query_id] = avg_precision_at_k(np.array(current_query_res["label"]), np.array(current_query_res["predicted_relevance"]), k)
    print(query_id,":",avg_precision_tq[query_id])'''

'avg_precision_tq = {}\nfor query_id, query in enumerate(test_queries, start=1):\n\n    current_query = query_id\n    current_query_res = search_results_tq[search_results_tq["query_id"] == current_query]\n    avg_precision_tq[query_id] = avg_precision_at_k(np.array(current_query_res["label"]), np.array(current_query_res["predicted_relevance"]), k)\n    print(query_id,":",avg_precision_tq[query_id])'

**3.2.4 F1-Score @K**


In [258]:
'''def f1score_at_k(precision, recall):
    """
    Calculate the F1 score.

    Parameters:
    - precision (float): Precision score.
    - recall (float): Recall score.

    Returns:
    - f1 (float): F1 score.
    """
    # Check if both precision and recall are greater than zero to avoid division by zero
    if precision + recall == 0:
        return 0.0
    # Calculate the F1 score
    return 2 * (precision * recall) / (precision + recall)'''

'def f1score_at_k(precision, recall):\n    """\n    Calculate the F1 score.\n\n    Parameters:\n    - precision (float): Precision score.\n    - recall (float): Recall score.\n\n    Returns:\n    - f1 (float): F1 score.\n    """\n    # Check if both precision and recall are greater than zero to avoid division by zero\n    if precision + recall == 0:\n        return 0.0\n    # Calculate the F1 score\n    return 2 * (precision * recall) / (precision + recall)'

Test for provided queries:

In [259]:
'''f1score_q = {}

for query_id, query in enumerate(queries, start=1):

    current_query = query_id
    current_query_res = search_results_q[search_results_q["query_id"] == current_query]

    f1score_q[query_id] = f1score_at_k(precision_q[query_id], recall_q[query_id])
    print(query_id,":",f1score_q[query_id])'''

'f1score_q = {}\n\nfor query_id, query in enumerate(queries, start=1):\n\n    current_query = query_id\n    current_query_res = search_results_q[search_results_q["query_id"] == current_query]\n\n    f1score_q[query_id] = f1score_at_k(precision_q[query_id], recall_q[query_id])\n    print(query_id,":",f1score_q[query_id])'

Test for created test_queries:

In [260]:
'''f1score_tq = {}

for query_id, query in enumerate(test_queries, start=1):

    current_query = query_id
    current_query_res = search_results_tq[search_results_q["query_id"] == current_query]

    f1score_tq[query_id] = f1score_at_k(precision_tq[query_id], recall_tq[query_id])
    print(query_id,":",f1score_tq[query_id])'''

'f1score_tq = {}\n\nfor query_id, query in enumerate(test_queries, start=1):\n\n    current_query = query_id\n    current_query_res = search_results_tq[search_results_q["query_id"] == current_query]\n\n    f1score_tq[query_id] = f1score_at_k(precision_tq[query_id], recall_tq[query_id])\n    print(query_id,":",f1score_tq[query_id])'

**3.2.5 Mean Average Precision(MAP)**


In [261]:
'''# extracted from professor's practise lab file

def map_at_k(search_res, k=10):
    ''
    Parameters
    ----------
    search_res: search results dataset containing:
        q_id: query id.
        doc_id: document id.
        predicted_relevance: relevance predicted through LightGBM.
        y_true: actual score of the document for the query (ground truth).

    Returns
    -------
    mean average precision @k : float
    ''
    avp = []
    for q in search_res["query_id"].unique():  #loop over all query id
        curr_data = search_res[search_res["query_id"] == q]  # select data for current query
        avp.append(avg_precision_at_k(np.array(curr_data["label"]), np.array(curr_data["predicted_relevance"]),
                                      k))  #append average precision for current query
    return np.sum(avp) / len(avp), avp  # return mean average precision'''

'# extracted from professor\'s practise lab file\n\ndef map_at_k(search_res, k=10):\n    \'\'\n    Parameters\n    ----------\n    search_res: search results dataset containing:\n        q_id: query id.\n        doc_id: document id.\n        predicted_relevance: relevance predicted through LightGBM.\n        y_true: actual score of the document for the query (ground truth).\n\n    Returns\n    -------\n    mean average precision @k : float\n    \'\'\n    avp = []\n    for q in search_res["query_id"].unique():  #loop over all query id\n        curr_data = search_res[search_res["query_id"] == q]  # select data for current query\n        avp.append(avg_precision_at_k(np.array(curr_data["label"]), np.array(curr_data["predicted_relevance"]),\n                                      k))  #append average precision for current query\n    return np.sum(avp) / len(avp), avp  # return mean average precision'

mAP@10 for the two queries provided:

In [262]:
'''map_k_q, avp_q = map_at_k(search_results_q, 10)
map_k_q'''

'map_k_q, avp_q = map_at_k(search_results_q, 10)\nmap_k_q'

mAP@10 for the test_queries created:

In [263]:
'''map_k_tq, avp_tq = map_at_k(search_results_tq, 10)
map_k_tq'''

'map_k_tq, avp_tq = map_at_k(search_results_tq, 10)\nmap_k_tq'

**3.2.6 Mean Reciprocal Rank(MRR)**


In [264]:
'''# extracted from professor's practise lab file

def rr_at_k(doc_score, y_score, k=10):
    """
    Parameters
    ----------
    doc_score: Ground truth (true relevance labels).
    y_score: Predicted scores.
    k : number of doc to consider.

    Returns
    -------
    Reciprocal Rank for qurrent query
    """
    # get the list of indexes of the predicted score sorted in descending order.
    order = np.argsort(y_score)[::-1]
    # sort the actual relevance label of the documents based on predicted score(hint: np.take) and take first k.
    doc_score = np.take(doc_score, order[:k])
    if np.sum(doc_score) == 0:  # if there are not relevant doument return 0
        return 0
    # hint: to get the position of the first relevant document use "np.argmax"
    return 1 / (np.argmax(doc_score == 1) + 1)'''

'# extracted from professor\'s practise lab file\n\ndef rr_at_k(doc_score, y_score, k=10):\n    """\n    Parameters\n    ----------\n    doc_score: Ground truth (true relevance labels).\n    y_score: Predicted scores.\n    k : number of doc to consider.\n\n    Returns\n    -------\n    Reciprocal Rank for qurrent query\n    """\n    # get the list of indexes of the predicted score sorted in descending order.\n    order = np.argsort(y_score)[::-1]\n    # sort the actual relevance label of the documents based on predicted score(hint: np.take) and take first k.\n    doc_score = np.take(doc_score, order[:k])\n    if np.sum(doc_score) == 0:  # if there are not relevant doument return 0\n        return 0\n    # hint: to get the position of the first relevant document use "np.argmax"\n    return 1 / (np.argmax(doc_score == 1) + 1)'

In [265]:
'''y_true = np.array([0, 1, 0, 1, 1])
score = np.array([0.9, 0.5, 0.6, 0.7, 0.2])
rr_at_k(y_true, score, 10)'''

'y_true = np.array([0, 1, 0, 1, 1])\nscore = np.array([0.9, 0.5, 0.6, 0.7, 0.2])\nrr_at_k(y_true, score, 10)'

Test for provided queries:

In [266]:
'''rr_at_k_q = {}
for query_id, query in enumerate(queries, start=1):

    current_query = query_id
    current_query_res = search_results_q[search_results_q["query_id"] == current_query]
    current_query_res.sort_values("predicted_relevance", ascending=False).head(5)
    labels = np.array(search_results_q[search_results_q['query_id'] == current_query]["label"])
    scores = np.array(search_results_q[search_results_q['query_id'] == current_query]["predicted_relevance"])
    rr_at_k_q[query_id] = np.round(rr_at_k(labels, scores, k),4)
    print(query_id,":",rr_at_k_q[query_id])'''

'rr_at_k_q = {}\nfor query_id, query in enumerate(queries, start=1):\n\n    current_query = query_id\n    current_query_res = search_results_q[search_results_q["query_id"] == current_query]\n    current_query_res.sort_values("predicted_relevance", ascending=False).head(5)\n    labels = np.array(search_results_q[search_results_q[\'query_id\'] == current_query]["label"])\n    scores = np.array(search_results_q[search_results_q[\'query_id\'] == current_query]["predicted_relevance"])\n    rr_at_k_q[query_id] = np.round(rr_at_k(labels, scores, k),4)\n    print(query_id,":",rr_at_k_q[query_id])'

Test for created test_queries:

In [267]:
'''rr_at_k_tq = {}
for query_id, query in enumerate(test_queries, start=1):

    current_query = query_id
    current_query_res = search_results_tq[search_results_tq["query_id"] == current_query]
    current_query_res.sort_values("predicted_relevance", ascending=False).head(5)
    labels = np.array(search_results_tq[search_results_tq['query_id'] == current_query]["label"])
    scores = np.array(search_results_tq[search_results_tq['query_id'] == current_query]["predicted_relevance"])
    rr_at_k_tq[query_id] = np.round(rr_at_k(labels, scores, k),4)
    print(query_id,":",rr_at_k_tq[query_id])'''

'rr_at_k_tq = {}\nfor query_id, query in enumerate(test_queries, start=1):\n\n    current_query = query_id\n    current_query_res = search_results_tq[search_results_tq["query_id"] == current_query]\n    current_query_res.sort_values("predicted_relevance", ascending=False).head(5)\n    labels = np.array(search_results_tq[search_results_tq[\'query_id\'] == current_query]["label"])\n    scores = np.array(search_results_tq[search_results_tq[\'query_id\'] == current_query]["predicted_relevance"])\n    rr_at_k_tq[query_id] = np.round(rr_at_k(labels, scores, k),4)\n    print(query_id,":",rr_at_k_tq[query_id])'

Compute the MRR@K for k=3,5,and 10.

Test for provided queries

In [268]:
'''mrr_q = {}
for k in [3, 5, 10]:
    RRs = []
    for q in search_results_q['query_id'].unique():  # loop over all query ids
        labels = np.array(search_results_q[search_results_q['query_id'] == q]["label"])  # get labels for current query
        scores = np.array(search_results_q[search_results_q['query_id'] == q]["predicted_relevance"])  # get predicted score for current query
        RRs.append(rr_at_k(labels, scores, k))  # append RR for current query
    mrr_q[k] = np.round(float(sum(RRs) / len(RRs)), 4)  # Mean RR at current k

mrr_q'''

'mrr_q = {}\nfor k in [3, 5, 10]:\n    RRs = []\n    for q in search_results_q[\'query_id\'].unique():  # loop over all query ids\n        labels = np.array(search_results_q[search_results_q[\'query_id\'] == q]["label"])  # get labels for current query\n        scores = np.array(search_results_q[search_results_q[\'query_id\'] == q]["predicted_relevance"])  # get predicted score for current query\n        RRs.append(rr_at_k(labels, scores, k))  # append RR for current query\n    mrr_q[k] = np.round(float(sum(RRs) / len(RRs)), 4)  # Mean RR at current k\n\nmrr_q'

Test for test_queries

In [269]:
'''mrr_tq = {}
for k in [3, 5, 10]:
    RRs = []
    for q in search_results_tq['query_id'].unique():  # loop over all query ids
        labels = np.array(search_results_tq[search_results_tq['query_id'] == q]["label"])  # get labels for current query
        scores = np.array(search_results_tq[search_results_tq['query_id'] == q]["predicted_relevance"])  # get predicted score for current query
        RRs.append(rr_at_k(labels, scores, k))  # append RR for current query
    mrr_tq[k] = np.round(float(sum(RRs) / len(RRs)), 4)  # Mean RR at current k

mrr_tq'''

'mrr_tq = {}\nfor k in [3, 5, 10]:\n    RRs = []\n    for q in search_results_tq[\'query_id\'].unique():  # loop over all query ids\n        labels = np.array(search_results_tq[search_results_tq[\'query_id\'] == q]["label"])  # get labels for current query\n        scores = np.array(search_results_tq[search_results_tq[\'query_id\'] == q]["predicted_relevance"])  # get predicted score for current query\n        RRs.append(rr_at_k(labels, scores, k))  # append RR for current query\n    mrr_tq[k] = np.round(float(sum(RRs) / len(RRs)), 4)  # Mean RR at current k\n\nmrr_tq'

**3.2.7 Normalized Discounted Cumulative Gain(NDCG)**

In [270]:
'''# extracted from professor's practise lab file

def dcg_at_k(y_true, y_score, k=10):
    # get the list of indexes of the predicted score sorted in descending order.
    order = np.argsort(y_score)[::-1]
    # sort the actual relevance label of the documents based on predicted score(hint: np.take) and take first k.
    y_true = np.take(y_true, order[:k])
    gain = 2 ** y_true - 1  # Compute gain (use formula 7 above)
    discounts = np.log2(np.arange(len(y_true)) + 2)  # Compute denominator
    return np.sum(gain / discounts)  #return dcg@k


def ndcg_at_k(y_true, y_score, k=10):
    dcg_max = dcg_at_k(y_true, y_true, k)
    if not dcg_max:
        return 0
    return np.round(dcg_at_k(y_true, y_score, k) / dcg_max, 4)'''

"# extracted from professor's practise lab file\n\ndef dcg_at_k(y_true, y_score, k=10):\n    # get the list of indexes of the predicted score sorted in descending order.\n    order = np.argsort(y_score)[::-1]\n    # sort the actual relevance label of the documents based on predicted score(hint: np.take) and take first k.\n    y_true = np.take(y_true, order[:k])\n    gain = 2 ** y_true - 1  # Compute gain (use formula 7 above)\n    discounts = np.log2(np.arange(len(y_true)) + 2)  # Compute denominator\n    return np.sum(gain / discounts)  #return dcg@k\n\n\ndef ndcg_at_k(y_true, y_score, k=10):\n    dcg_max = dcg_at_k(y_true, y_true, k)\n    if not dcg_max:\n        return 0\n    return np.round(dcg_at_k(y_true, y_score, k) / dcg_max, 4)"

##### the  𝑁𝐷𝐶𝐺@10 for each query

Test for provided queries:

In [271]:
'''ndcg_q = {}
for query_id, query in enumerate(queries, start=1):

    current_query = query_id
    k = 10
    labels = np.array(search_results_q[search_results_q['query_id'] == current_query]["label"])
    scores = np.array(search_results_q[search_results_q['query_id'] == current_query]["predicted_relevance"])
    ndcg_q[query_id] = np.round(ndcg_at_k(labels, scores, k), 4)
    print(query_id,":",ndcg_q[query_id])
'''

'ndcg_q = {}\nfor query_id, query in enumerate(queries, start=1):\n\n    current_query = query_id\n    k = 10\n    labels = np.array(search_results_q[search_results_q[\'query_id\'] == current_query]["label"])\n    scores = np.array(search_results_q[search_results_q[\'query_id\'] == current_query]["predicted_relevance"])\n    ndcg_q[query_id] = np.round(ndcg_at_k(labels, scores, k), 4)\n    print(query_id,":",ndcg_q[query_id])\n'

Test for created test_queries:

In [272]:
'''ndcg_tq = {}
for query_id, query in enumerate(test_queries, start=1):

    current_query = query_id
    k = 10
    labels = np.array(search_results_tq[search_results_tq['query_id'] == current_query]["label"])
    scores = np.array(search_results_tq[search_results_tq['query_id'] == current_query]["predicted_relevance"])
    ndcg_tq[query_id] = np.round(ndcg_at_k(labels, scores, k), 4)
    print(query_id,":",ndcg_tq[query_id])'''

'ndcg_tq = {}\nfor query_id, query in enumerate(test_queries, start=1):\n\n    current_query = query_id\n    k = 10\n    labels = np.array(search_results_tq[search_results_tq[\'query_id\'] == current_query]["label"])\n    scores = np.array(search_results_tq[search_results_tq[\'query_id\'] == current_query]["predicted_relevance"])\n    ndcg_tq[query_id] = np.round(ndcg_at_k(labels, scores, k), 4)\n    print(query_id,":",ndcg_tq[query_id])'

the average  𝑁𝐷𝐶𝐺@10  considering all queries/rankings.

Test for provided queries:

In [273]:
'''ndcgs = []
k = 10
for q in search_results_q['query_id'].unique():
    labels = np.array(search_results_q[search_results_q['query_id'] == q]["label"])
    scores = np.array(search_results_q[search_results_q['query_id'] == q]["predicted_relevance"])
    ndcgs.append(np.round(ndcg_at_k(labels, scores, k), 4))

avg_ndcg = np.round(float(sum(ndcgs) / len(ndcgs)), 4)
print("Average ndcg@{} for provided queries: {}".format(k, avg_ndcg))
'''

'ndcgs = []\nk = 10\nfor q in search_results_q[\'query_id\'].unique():\n    labels = np.array(search_results_q[search_results_q[\'query_id\'] == q]["label"])\n    scores = np.array(search_results_q[search_results_q[\'query_id\'] == q]["predicted_relevance"])\n    ndcgs.append(np.round(ndcg_at_k(labels, scores, k), 4))\n\navg_ndcg = np.round(float(sum(ndcgs) / len(ndcgs)), 4)\nprint("Average ndcg@{} for provided queries: {}".format(k, avg_ndcg))\n'

Test for created test_queries:

In [274]:
'''ndcgs = []
k = 10
for q in search_results_tq['query_id'].unique():
    labels = np.array(search_results_tq[search_results_tq['query_id'] == q]["label"])
    scores = np.array(search_results_tq[search_results_tq['query_id'] == q]["predicted_relevance"])
    ndcgs.append(np.round(ndcg_at_k(labels, scores, k), 4))

avg_ndcg = np.round(float(sum(ndcgs) / len(ndcgs)), 4)
print("Average ndcg@{} for test_queries: {}".format(k, avg_ndcg))'''

'ndcgs = []\nk = 10\nfor q in search_results_tq[\'query_id\'].unique():\n    labels = np.array(search_results_tq[search_results_tq[\'query_id\'] == q]["label"])\n    scores = np.array(search_results_tq[search_results_tq[\'query_id\'] == q]["predicted_relevance"])\n    ndcgs.append(np.round(ndcg_at_k(labels, scores, k), 4))\n\navg_ndcg = np.round(float(sum(ndcgs) / len(ndcgs)), 4)\nprint("Average ndcg@{} for test_queries: {}".format(k, avg_ndcg))'

##**All Results**

For provided queries:

In [275]:
'''print("Provided Queries: \n")
for query_id, query in enumerate(queries, start=1):
  print("query_id: ",query_id,": ",query,"\n")
  print("precision: ",precision_q[query_id])
  print("recall: ",recall_q[query_id])
  print("avg_precision:",avg_precision_q[query_id])
  print("F1_score: ",f1score_q[query_id])
  print("map_k_q: ",map_k_q)
  print("rr_at_k:",rr_at_k_q[query_id])
  print("ndcd: ",ndcg_q[query_id],"\n")'''

'print("Provided Queries: \n")\nfor query_id, query in enumerate(queries, start=1):\n  print("query_id: ",query_id,": ",query,"\n")\n  print("precision: ",precision_q[query_id])\n  print("recall: ",recall_q[query_id])\n  print("avg_precision:",avg_precision_q[query_id])\n  print("F1_score: ",f1score_q[query_id])\n  print("map_k_q: ",map_k_q)\n  print("rr_at_k:",rr_at_k_q[query_id])\n  print("ndcd: ",ndcg_q[query_id],"\n")'

For created test_queries:

In [276]:
'''print("Test_queries: \n")
for query_id, query in enumerate(test_queries, start=1):
  print("query_id: ",query_id,": ",query,"\n")
  print("precision: ",precision_tq[query_id])
  print("recall: ",recall_tq[query_id])
  print("avg_precision:",avg_precision_tq[query_id])
  print("F1_score: ",f1score_tq[query_id])
  print("map_k_q: ",map_k_tq)
  print("rr_at_k:",rr_at_k_tq[query_id])
  print("ndcd: ",ndcg_tq[query_id],"\n")'''

'print("Test_queries: \n")\nfor query_id, query in enumerate(test_queries, start=1):\n  print("query_id: ",query_id,": ",query,"\n")\n  print("precision: ",precision_tq[query_id])\n  print("recall: ",recall_tq[query_id])\n  print("avg_precision:",avg_precision_tq[query_id])\n  print("F1_score: ",f1score_tq[query_id])\n  print("map_k_q: ",map_k_tq)\n  print("rr_at_k:",rr_at_k_tq[query_id])\n  print("ndcd: ",ndcg_tq[query_id],"\n")'

In [277]:
#### We have setted a random seed initially, but it might not work properly and these results may vary

Query 1: "protest indian"

With a precision of 0.3 and recall of 1.0, all relevant documents were retrieved, although only 30% of the results were relevant. This suggests moderate retrieval effectiveness with some non-relevant results mixed in. The F1 score of 0.46 reflects this balance, while the average precision of 0.29 implies that relevant documents may not be consistently ranked at the top. MAP and NDCG scores (0.61 and 0.52, respectively) indicate a reasonable ranking of relevant documents, although some improvement in ordering would enhance relevance.

Query 2: "protest farmer"

This query achieves a slightly higher precision of 0.4 with full recall, meaning that 40% of the retrieved documents were relevant. The F1 score of 0.57 reflects a more favorable precision-recall balance. An RR_at_k of 1.0 indicates that a relevant document was retrieved in the first position, enhancing user satisfaction. An NDCG of 0.76 shows that relevant documents are generally ranked effectively, improving retrieval quality for this query.

Query 3: "support farmer"

With precision at 0.6 and recall at 1.0, this query demonstrates strong performance, achieving an F1 score of 0.75. The average precision of 0.62 shows that relevant documents are ranked favorably, with high positions in the results. The RR_at_k at 0.5 and NDCG of 0.76 further confirm that relevant documents are effectively retrieved and ranked, reflecting a well-balanced retrieval for this query.

Query 4: "free india"

This query maintains a strong balance with precision at 0.2 and full recall, yielding an F1 score of 0.33. Despite lower precision, the high average precision of 1.0 and an NDCG of 1.0 indicate that the relevant documents are consistently ranked at the top. RR_at_k remains 1.0, demonstrating excellent performance in surfacing relevant documents early in the ranking.

Query 5: "universitat pompeu fabra"

For this query, no relevant documents were retrieved, resulting in a precision and recall of 0. This affects metrics like MAP, RR_at_k, and NDCG, which reflect a lack of relevant retrieval for this query. Since this query acts as a control without expected relevant documents in the index, the results align reasonably with expectations.


*3.3 Choose one vector representation, TF-IDF or Word2Vec, and represent the tweets in a two-dimensional scatter plot through the T-SNE (T-distributed Stochastic Neighbor Embedding) algorithm. To do so, you may need first to represent the word as a vector, and then the tweet, i.e., resulting as the average value over the words involved. Any other option rather than T-SNE may be used, but needs to be justified.*

**HINT:** *You don’t have to know all the theoretical details used in T-SNE, just use the proper library and generate the output and play with it.*

*Also, you can choose to perform an alternative method to generate a 2-dimensional representation for the word embeddings (like PCA).*

*Here are some TSNE examples which may be good guidelines for the task:*

1. [https://towardsdatascience.com/google-news-and-leo-tolstoy-visualizing-word2vec-word-embeddings-with-t-sne-11558d8bd4](https://)

1. [https://towardsdatascience.com/visualizing-word-embedding-with-pca-and-t-sne-961a692509f5](https://)

2. [https://stackoverflow.com/questions/40581010how-to-run-tsne-on-word2vec-creat
 ed-from-gensim](https://)


In [278]:
'''tweets = tweet_info_df['Tweet'].astype(str).tolist()  # convert to list of strings

# vectorize the tweets using TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=30)  # limit to top 30 features
tfidf_matrix = tfidf_vectorizer.fit_transform(tweets)

# apply T-SNE for dimensionality reduction to 2D
perplexity_value = min(30, len(tweets) - 1)  # Set perplexity to a safe value less than the number of samples
tsne = TSNE(n_components=2, random_state=42, perplexity=perplexity_value, max_iter=300)
tweet_embeddings_2d = tsne.fit_transform(tfidf_matrix.toarray())

# 2D scatter plot
plt.figure(figsize=(12, 8))
plt.scatter(tweet_embeddings_2d[:, 0], tweet_embeddings_2d[:, 1], c='blue', edgecolor='k', s=50)
plt.title('2D T-SNE Visualization of Tweets (TF-IDF Representation)', fontsize=15)
plt.xlabel('T-SNE Component 1')
plt.ylabel('T-SNE Component 2')
plt.grid(True)
plt.show()'''

"tweets = tweet_info_df['Tweet'].astype(str).tolist()  # convert to list of strings\n\n# vectorize the tweets using TF-IDF\ntfidf_vectorizer = TfidfVectorizer(max_features=30)  # limit to top 30 features\ntfidf_matrix = tfidf_vectorizer.fit_transform(tweets)\n\n# apply T-SNE for dimensionality reduction to 2D\nperplexity_value = min(30, len(tweets) - 1)  # Set perplexity to a safe value less than the number of samples\ntsne = TSNE(n_components=2, random_state=42, perplexity=perplexity_value, max_iter=300)\ntweet_embeddings_2d = tsne.fit_transform(tfidf_matrix.toarray())\n\n# 2D scatter plot\nplt.figure(figsize=(12, 8))\nplt.scatter(tweet_embeddings_2d[:, 0], tweet_embeddings_2d[:, 1], c='blue', edgecolor='k', s=50)\nplt.title('2D T-SNE Visualization of Tweets (TF-IDF Representation)', fontsize=15)\nplt.xlabel('T-SNE Component 1')\nplt.ylabel('T-SNE Component 2')\nplt.grid(True)\nplt.show()"

In the previous plot we can see a graphic which represents tweets based on their TF-IDF vectors, each dot represents an individual tweet positioned according to its word frequency and relevance. Since we can only observe one cluster, this could mean that the tweets may not contain strongly differentiated content based on their TF-IDF scores, or it could be due to T-SNE’s parameter settings. The majority of the points are concentrated near the center, indicating that many tweets share similar word distributions. This makes sense when you consider that they are all about the same topic: the farm protests in India.

#**PART 3: Ranking**

#**2. Ranking score:**

**Given a query, we want to get the top-20 documents related to the query.**

**GOAL:**

**Find all the documents that contain all the words in the query and sort them by their relevance with regard to the query.**

**SCORE:**

##**2.1. You’re asked to provide 2 different ways of ranking:**

  **2.1.1 a. TF-IDF + cosine similarity: Classical scoring, we have also seen during the practical labs**



In [279]:
tweets = tweet_info_df['Tweet'].astype(str).tolist()

In [280]:
def rank_with_tfidf(query, documents):
    """
    Ranks documents based on a query using TF-IDF and cosine similarity.

    Parameters:
    - query: The query string.
    - documents: A list of document strings.

    Returns:
    - A list of (document, score) tuples, ranked by relevance to the query.
    """
    # Combine query and documents for TF-IDF computation
    all_texts = [query] + documents
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(all_texts)

    # Calculate cosine similarity between the query vector and document vectors
    query_vector = tfidf_matrix[0]
    doc_vectors = tfidf_matrix[1:]
    cosine_similarities = cosine_similarity(query_vector, doc_vectors).flatten()

    # Rank documents by similarity score
    ranked_docs = sorted(zip(documents, cosine_similarities), key=lambda x: x[1], reverse=True)
    return ranked_docs[:20]

In [281]:
for query in test_queries:
    ranked_docs_tfidf = rank_with_tfidf(query, tweets)

    # Convert ranked_docs_tfidf (list of tuples) into a DataFrame
    df_ranked = pd.DataFrame(ranked_docs_tfidf, columns=['Tweet', 'Rating'])

    # Display the DataFrame with query as context
    print(f"Query: {query}")
    display(df_ranked)
    print('__________________________________________________________________________')
    print()

Query: protest indian


,Tweet,Rating
0,indian protest,1.000000
1,indian protest,1.000000
2,indian protest,1.000000
3,indian protest,1.000000
4,indian protest,1.000000
5,indian protest,1.000000
6,indian protest,1.000000
7,indian protest,1.000000
8,indian protest,1.000000
9,indian protest,1.000000


__________________________________________________________________________

Query: protest farmer


,Tweet,Rating
0,farmer protest,1.000000
1,farmer protest,1.000000
2,farmer protest,1.000000
3,farmer protest,1.000000
4,farmer protest,1.000000
5,farmer protest,1.000000
6,farmer protest,1.000000
7,farmer protest,1.000000
8,farmer protest,1.000000
9,farmer protest,1.000000


__________________________________________________________________________

Query: support farmer


...

__________________________________________________________________________

Query: free india 


,Tweet,Rating
0,free,0.876111
1,free,0.876111
2,free,0.876111
3,free,0.876111
4,free,0.876111
5,free protestor free journalist free women free...,0.752484
6,free peopl,0.731958
7,free ranjit singh free nodeep kaur free shiv k...,0.699501
8,free countri,0.681730
9,free kisanektamorcha,0.648264


__________________________________________________________________________

Query: universitat pompeu fabra


,Tweet,Rating
0,world progress indian polic govt still tri tak...,0.0
1,kisanektamorcha farmer constantli distroy crop...,0.0
2,reallyswara rohinisgh watch full video,0.0
3,mandeeppunia1 watch full video,0.0
4,mandeeppunia1 watch full video,0.0
5,anoth farmer malkeet singh mahilpur hoshiarpur...,0.0
6,akshaykumar hi tell boss thank,0.0
7,taapse watch full video,0.0
8,despit increas must increas petroleummin pmoindia,0.0
9,mockeri menac charg,0.0


__________________________________________________________________________



**2.1.2 b. Your-Score + cosine similarity: Here the task is to create a new score, and it’s up to you to create a new one.**


### Approach and Procedure

For this implementation, we will use **Item-based Collaborative Filtering**, adapted to the context of tweets and vocabulary. This method considers items (words in tweets) that a user has already “consumed” (that appear in relevant tweets) and recommends other items that have been rated similarly.

### Recommendation Process
To recommend words that might be important in a search query, we follow these steps:

1. **Calculate the Popularity of Each Tweet**:  
   We calculate a popularity score for each tweet using the formula:

   $$
   \text{tweet_mark} = \log(\text{Likes} + \text{Retweets} + 1)
   $$

   where:
   - `Likes`: The total number of likes on the tweet.
   - `Retweets`: The total number of retweets on the tweet.

   We filter tweets with a **minimum popularity score** to keep only the most relevant ones.

2. **Filter Tweets Based on the Query**:  
   We filter tweets that contain all the words in the query, allowing us to focus on the tweets most relevant to that query.

3. **Vector Representation of Tweets and Query with TF-IDF**:  
   We convert the text of the tweets into a TF-IDF vector matrix, which assigns weights to words based on their frequency in each tweet and their rarity across the entire set of tweets.

4. **Cosine Similarity Between the Query and Tweets**:  
   We calculate the **cosine similarity** between the query vector and the vectors of each filtered tweet. This helps identify tweets whose word composition is most similar to the query.

5. **Calculate the Custom Score (Your-Score)**:  
   We combine the popularity score and cosine similarity into a custom metric, called `our_score`, calculated as:

   $$
   \text{our_score} = \text{tweet_mark} \times \text{cosine_similarity}
   $$

   This score weighs tweets that are not only popular but also relevant to the query in terms of word composition.

6. **Top 20 Recommended Tweets**:  
   Finally, we sort the tweets by `our_score` and select the top 20 as recommendations, providing a list of tweets that are both popular and relevant to the query.

This approach allows us to identify the tweets that best respond to a query in terms of thematic relevance and popularity. Additionally, by calculating cosine similarity with the TF-IDF model, we prioritize tweets whose vocabulary more closely resembles that of the user’s query.

In [282]:
# Calculate the popularity score for each tweet
def calcular_nota(row):
    return math.log(row['Likes'] + row['Retweets'] + 1)

def get_top20_our_score(query, tweets_df):
    """
    query: text of the query
    tweets_df: DataFrame of tweets, must include 'Likes', 'Retweets', and 'Tweet' columns

    This function calculates a custom score based on popularity
    (Likes and Retweets) and cosine similarity with the query, and returns the
    20 most relevant tweets.
    """
    # Calculate the popularity score for each tweet
    tweets_df['nota_tweet'] = tweets_df.apply(calcular_nota, axis=1)

    # Filter the tweets with a minimum score of 3
    df_important_tweets = tweets_df[tweets_df.nota_tweet >= 3].copy()

    # Filter the tweets that contain all the words in the query
    query_words = set(query.lower().split())
    df_important_tweets['contains_query'] = df_important_tweets['Tweet'].apply(lambda x: query_words.issubset(set(x.lower().split())))
    df_filtered = df_important_tweets[df_important_tweets['contains_query']].copy()

    if df_filtered.empty:
      print('No important docs found')
      return df_filtered

    # Create the TF-IDF representation for the set of tweets and the query
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(df_filtered['Tweet'])

    # Vector of the query
    query_tfidf = tfidf_vectorizer.transform([query])

    # Calculate the cosine similarity between the query and the tweets
    cosine_scores = cosine_similarity(query_tfidf, tfidf_matrix).flatten()
    df_filtered['cosine_similarity'] = cosine_scores

    # Define the custom score (Your-Score) combined with the cosine similarity
    df_filtered['our_score'] = df_filtered['nota_tweet'] * df_filtered['cosine_similarity']

    # Sort and display the top-20 according to the Your-Score
    top_20_custom = df_filtered.sort_values(by='our_score', ascending=False).head(20)

    return top_20_custom[['Tweet', 'Likes', 'Retweets', 'nota_tweet', 'cosine_similarity', 'our_score']]


In [283]:
# Create the DataFrame of already processed tweets
df = tweet_info_df.copy()

# Execute the function with the query and the dataframe of tweets
for query in test_queries:
    top_20_tweets = get_top20_our_score(query, df)
    # Display the top 20 tweets based on our custom score
    print('Query:', query)
    display(top_20_tweets.reset_index())
    print('__________________________________________________________________________________________________________________')
    print()

Query: protest indian


,index,Tweet,Likes,Retweets,nota_tweet,cosine_similarity,our_score
0,34406,indian largest protest world support farmer,706,720,7.263330,0.319404,2.319936
1,34197,indian largest protest world support farmer,232,254,6.188264,0.319404,1.976556
2,80224,indian farmer protest matter british indian,79,34,4.736198,0.373701,1.769922
3,34291,indian largest protest world support farmer,24,46,4.262680,0.319404,1.361517
4,33581,indian largest protest world support farmer,22,37,4.094345,0.319404,1.307750
5,33673,indian largest protest world support farmer,33,24,4.060443,0.319404,1.296922
6,33676,indian largest protest world support farmer,12,27,3.688879,0.319404,1.178243
7,44296,indian farmer protest explain part 3,86,43,4.867534,0.227675,1.108214
8,33829,indian largest protest world support farmer,9,18,3.332205,0.319404,1.064319
9,32025,indian largest protest world support farmer,11,12,3.178054,0.319404,1.015083


__________________________________________________________________________________________________________________

Query: protest farmer


,index,Tweet,Likes,Retweets,nota_tweet,cosine_similarity,our_score
0,32032,keep support peac farmer protest,3679,1554,8.562931,0.234164,2.005129
1,34406,indian largest protest world support farmer,706,720,7.263330,0.205200,1.490438
2,305,great illustr protest farmer protest delhi,235,50,5.655992,0.237512,1.343368
3,97922,farmer protest germani,149,62,5.356586,0.250239,1.340428
4,34197,indian largest protest world support farmer,232,254,6.188264,0.205200,1.269834
5,116255,farmer protest turn point india time,1055,439,7.309881,0.155920,1.139759
6,64112,countless human right org farmer amp activist ...,2781,1201,8.289791,0.125611,1.041290
7,34472,largest protest world support farmer,31,47,4.369448,0.230600,1.007593
8,91123,farmer protest germani,28,13,3.737670,0.250239,0.935312
9,93623,farmer protest across india,21,16,3.637586,0.254981,0.927517


__________________________________________________________________________________________________________________

Query: support farmer


...

__________________________________________________________________________________________________________________

Query: free india 


,index,Tweet,Likes,Retweets,nota_tweet,cosine_similarity,our_score
0,64492,india target young women silenc dissent amp mu...,7244,3230,9.256747,0.099174,0.918032
1,26708,attack free thought speech extend well beyond ...,47,12,4.094345,0.214090,0.876559
2,117395,modi want make slave free india come join farm...,208,246,6.120297,0.141741,0.867500
3,36231,piec evolv perhap largest mass mobilis postcol...,860,301,7.057898,0.121337,0.856384
4,110041,ani bku becom insan day day support dwindl ind...,141,7,5.003946,0.170719,0.854269
5,75386,disha ravi 21 yo climat activist arrest delhi ...,1975,962,7.985484,0.106643,0.851596
6,28216,largest mass mobilis postcoloni india histori ...,97,36,4.897840,0.148744,0.728523
7,69137,21yr climat activist arrest delhi polic tweet ...,122,24,4.990433,0.108033,0.539132
8,9940,doubt india free nehru gandhi dynasti pakistan...,35,28,4.158883,0.117445,0.488441
9,86524,pbnsindia insan modi regul free speech didnt r...,20,3,3.178054,0.146736,0.466336


__________________________________________________________________________________________________________________

No important docs found
Query: universitat pompeu fabra


,index,Tweet ID,Tweet,Date,Hashtags,Likes,Retweets,Url,Document ID,nota_tweet,contains_query


__________________________________________________________________________________________________________________



These results show the top 20 tweets for the query "protest farmer," ranked by a custom score that prioritizes both popularity and relevance to the query. The nota_tweet (popularity score) is calculated based on the number of likes and retweets, with higher values indicating more popular tweets. Additionally, the cosine_similarity measures how closely the content of each tweet matches the words in the query. The our_score combines these two factors, giving higher weight to tweets that are both popular and contextually relevant.

Thus, the tweets shown here are the ones that are not only highly liked and retweeted but also contain relevant terms from the search query. This approach ensures that the most relevant and engaging tweets for the query "protest farmer" are recommended.


**2.1.3 c. BM25**



In [284]:
def rank_with_bm25(query, documents, k1=1.5, b=0.75):
    """
    Ranks documents based on a query using BM25.

    Parameters:
    - query: The query string.
    - documents: A list of document strings.
    - k1: BM25 hyperparameter for term frequency scaling.
    - b: BM25 hyperparameter for document length normalization.

    Returns:
    - A list of (document, score) tuples, ranked by BM25 relevance to the query.
    """
    # Tokenize the query and documents
    query_terms = query.lower().split()
    doc_terms = [doc.lower().split() for doc in documents]

    # Compute document frequencies (df) and average document length
    doc_lengths = [len(doc) for doc in doc_terms]
    avg_doc_length = np.mean(doc_lengths)
    df = Counter(term for doc in doc_terms for term in set(doc))  # Document frequencies (unique terms in each doc)

    # Total number of documents (N)
    N = len(documents)

    # Compute BM25 score for each document
    def idf(term):
        """Inverse Document Frequency for a given term."""
        return math.log((N - df.get(term, 0) + 0.5) / (df.get(term, 0) + 0.5) + 1.0)

    def bm25_score(doc, query_terms):
        """Compute BM25 score for a single document and query."""
        score = 0
        doc_counter = Counter(doc)  # Term frequencies in the document
        for term in query_terms:
            if term in doc_counter:
                tf = doc_counter[term]
                score += idf(term) * (tf * (k1 + 1)) / (tf + k1 * (1 - b + b * len(doc) / avg_doc_length))
        return score

    # Rank documents by BM25 score
    bm25_scores = [(doc, bm25_score(doc, query_terms)) for doc in doc_terms]
    ranked_docs = sorted(bm25_scores, key=lambda x: x[1], reverse=True)

    return ranked_docs[:20]

In [285]:
for query in test_queries:
    ranked_docs_bm25 = rank_with_bm25(query, tweets)

    # Convert ranked_docs_tfidf (list of tuples) into a DataFrame
    df_ranked_BM25 = pd.DataFrame(ranked_docs_bm25, columns=['Tweet', 'Rating'])

    # Display the DataFrame with query as context
    print(f"Query: {query}")
    display(df_ranked_BM25)
    print('_____________________________________________________________')
    print()

Query: protest indian


,Tweet,Rating
0,"[indian, protest]",8.406879
1,"[indian, protest]",8.406879
2,"[indian, protest]",8.406879
3,"[indian, protest]",8.406879
4,"[indian, protest]",8.406879
5,"[indian, protest]",8.406879
6,"[indian, protest]",8.406879
7,"[indian, protest]",8.406879
8,"[indian, protest]",8.406879
9,"[indian, protest]",8.406879


_____________________________________________________________

Query: protest farmer


,Tweet,Rating
0,"[farmer, protest, four, month, farmer, protest...",6.114822
1,"[farmer, protest]",5.919814
2,"[farmer, protest]",5.919814
3,"[farmer, protest]",5.919814
4,"[farmer, protest]",5.919814
5,"[farmer, protest]",5.919814
6,"[farmer, protest]",5.919814
7,"[farmer, protest]",5.919814
8,"[farmer, protest]",5.919814
9,"[farmer, protest]",5.919814


_____________________________________________________________

Query: support farmer


...

_____________________________________________________________

Query: free india 


,Tweet,Rating
0,"[india, free, democraci, anymor]",9.589824
1,"[gurnamsinghbku, need, bjp, free, india]",9.078782
2,"[india, system, oppress, export, free]",9.078782
3,"[free, india, daughter, free, democraci, benga...",8.886636
4,"[free, protestor, free, journalist, free, wome...",8.808162
5,"[shame, twitter, suppress, free, speech, india]",8.619451
6,"[nobodi, free, everybodi, free]",8.437321
7,"[free, ranjit, singh, free, nodeep, kaur, free...",8.392751
8,"[free, india, daughter, free, democraci, citiz...",8.359671
9,"[antiitcelltask1, right, path, ambani, adani, ...",8.204361


_____________________________________________________________

Query: universitat pompeu fabra


,Tweet,Rating
0,"[world, progress, indian, polic, govt, still, ...",0
1,"[kisanektamorcha, farmer, constantli, distroy,...",0
2,"[reallyswara, rohinisgh, watch, full, video]",0
3,"[mandeeppunia1, watch, full, video]",0
4,"[mandeeppunia1, watch, full, video]",0
5,"[anoth, farmer, malkeet, singh, mahilpur, hosh...",0
6,"[akshaykumar, hi, tell, boss, thank]",0
7,"[taapse, watch, full, video]",0
8,"[despit, increas, must, increas, petroleummin,...",0
9,"[mockeri, menac, charg]",0


_____________________________________________________________



**2.1.4 Comparing TF-IDF and BM25**

**Explain how the ranking differs when using TF-IDF and BM25 and think about the pros and cons of using them.**

The ranking methods **TF-IDF** and **BM25** differ in how they assess and prioritize relevance, leading to notable differences in the ranking outcomes.

TF-IDF (Term Frequency-Inverse Document Frequency) ranks tweets based on word frequency within the tweet relative to its rarity across the entire corpus. In the results above, TF-IDF gives a high similarity score of 1.0 to tweets containing both "indian" and "protest," but it also assigns progressively lower scores to tweets with variations of these terms and other additional terms. TF-IDF's reliance on frequency can make it less sensitive to differences in term density within documents, which sometimes leads to similar scores for documents of varying lengths, provided they contain the query terms.

BM25, by contrast, is a probabilistic ranking model that not only considers term frequency and document frequency but also adjusts based on document length, rewarding shorter, query-dense tweets. This means BM25 generally assigns higher scores to tweets where query terms are more concentrated and frequent. For example, tweets containing only the essential query terms “indian” and “protest” have the highest BM25 score (8.40), while tweets with additional context score slightly lower. BM25’s ability to scale with document length and term saturation generally allows it to better handle longer texts with relevant terms dispersed throughout.

- **TF-IDF Pros**: Simple, computationally efficient, and effective for short, structured text. However, it lacks length normalization, which can make it less effective with documents of varying lengths.
- **TF-IDF Cons**: It may miss context, as it doesn’t account for term saturation or document length, potentially overvaluing infrequent terms and underweighting important frequent terms.

- **BM25 Pros**: More flexible in handling different text lengths and term frequencies, making it robust for ranking documents of varying lengths, like tweets with diverse content. This makes it highly effective in search tasks and suited for real-world datasets.
- **BM25 Cons**: Slightly more computationally intensive due to its length normalization and saturation adjustments.

In summary, **BM25** often provides more nuanced rankings, especially beneficial for longer and more complex documents, whereas **TF-IDF** can be effective for simpler, shorter texts, but may lack the same level of adaptability.

**Regarding your own score, justify the choice of the score (pros and cons). The only constraint you have is that the score needs to involve the tweets information regarding the popularity over the social network (number of likes, number of tweets, number of comments, etc...)**

   The Our Score + Cosine Similarity method combines popularity (likes and retweets) with semantic relevance of tweets using cosine similarity and TF-IDF. Its advantages include a balanced recommendation between popularity and relevance, effective filtering of irrelevant tweets, and ease of implementation. However, it relies too much on popularity, which can skew results towards popular content that may not be highly relevant. Additionally, it is limited to exact word similarity, failing to capture nuances such as synonyms or context, and it doesn't consider the user's previous interactions, reducing personalization.

##**2.2. Top-20**

**Return a top-20 list of documents for each of the 5 queries, using word2vec + cosine similarity.**

  **To use the word2vec, you need to generate the tweet representation, which here is expressed as a unique vector of the same dimension of the words, generated as the average of the vectors representing the words included in the tweet:**
  
  **Ex. “I won the election”**
  
  **Having a vector (generated through word2vec) representing each word, e.g. I=v_1, won = v_2, the = v_3, election = v_4, all of the same number of dimensions n, it is possible to represent the text above and generating a unique representation, by averaging v_1, v_2, v_3, v_4. The result will be a new vector v, of the same dimension n representing the text “I won the election”. Since it’s a tweet in our case we will talk about tweet2vec.**


We want to find tweets that match each query (for example, "Indian protest" or "support for farmers") by comparing the meaning of the query to that of each tweet. To do this, we’ll use *Word2Vec*, which converts words into numerical vectors that capture the “meaning” of each word in a space of numbers.

### Step by Step

1. **Load a Word2Vec model**:  
   First, we need a Word2Vec model, which is like a dictionary that provides a vector (a set of numbers) for each word. These numbers capture each word’s meaning in relation to others.

2. **Convert tweets to vectors ("tweet2vec")**:  
   Word2Vec gives a vector for each word, but we need a single vector for each tweet. To do this, we take the vector for each word in the tweet and compute an *average* of these vectors. The result is a vector that represents the overall meaning of the complete tweet.

3. **Convert the query to a vector**:  
   Just as we did for the tweets, we take each word in the query, obtain its vector with Word2Vec, and calculate the average. This way, we convert the entire query into a single vector.

4. **Measure similarity**:  
   Once we have the vector for the query and the vector for each tweet, we calculate the *cosine similarity* between them, which tells us how similar they are in terms of meaning. Then, we can sort the tweets by their similarity to the query and return the 20 most similar.

In [286]:
# train a Word2Vec model on the preprocessed tweet data
def train_word2vec_model(tweets):
    # split tweets into words
    processed_tweets = [tweet.split() for tweet in tweets]

    # train Word2Vec model
    model = Word2Vec(sentences=processed_tweets, vector_size=100, window=5, min_count=1, workers=4)

    return model

tweets = tweet_info_df['Tweet'].tolist()
model = train_word2vec_model(tweets)

# tweet to a vector by averaging its word vectors
def tweet2vec(tweet, model):
    words = tweet.split()
    word_vectors = []

    for word in words:
        if word in model.wv:
            word_vectors.append(model.wv[word])  #word vector from the model

    if len(word_vectors) > 0:
        return np.mean(word_vectors, axis=0)
    else:
        return np.zeros(model.vector_size)

# rank documents using Word2Vec and cosine similarity
def rank_with_word2vec(query, documents, model):
    query_vector = tweet2vec(query, model)
    doc_vectors = [tweet2vec(doc, model) for doc in documents]

    cosine_similarities = cosine_similarity([query_vector], doc_vectors).flatten()

    ranked_docs = sorted(zip(documents, cosine_similarities), key=lambda x: x[1], reverse=True)
    return ranked_docs[:20]

In [287]:
for query in test_queries:
    ranked_docs_w2v = rank_with_word2vec(query, tweets, model)
    df_ranked = pd.DataFrame(ranked_docs_w2v, columns=['Tweet', 'Rating'])
    print(f"Query: {query}")
    display(df_ranked)
    print('__________________________________________________________________________')
    print()

Query: protest indian


,Tweet,Rating
0,indian protest,1.000000
1,indian protest,1.000000
2,indian protest,1.000000
3,indian protest,1.000000
4,indian protest,1.000000
5,indian protest,1.000000
6,indian protest,1.000000
7,indian protest,1.000000
8,indian protest,1.000000
9,indian protest,1.000000


__________________________________________________________________________

Query: protest farmer


,Tweet,Rating
0,farmer protest,1.000000
1,farmer protest,1.000000
2,farmer protest,1.000000
3,farmer protest,1.000000
4,farmer protest,1.000000
5,farmer protest,1.000000
6,farmer protest,1.000000
7,farmer protest,1.000000
8,farmer protest,1.000000
9,farmer protest,1.000000


__________________________________________________________________________

Query: support farmer


...

__________________________________________________________________________

Query: free india 


,Tweet,Rating
0,murder democrat india,0.964012
1,india free democraci anymor,0.963622
2,british empir arrest freedom fighter today ind...,0.960892
3,gujarati kill india,0.959133
4,babagloc karunanundi india judiciari rotten ce...,0.958939
5,mother india reboot 202021,0.958194
6,vulner citizen india suffer,0.956468
7,call freedom speechexpress india,0.954295
8,democrat valu india troubl,0.953056
9,india system oppress export free,0.951474


__________________________________________________________________________

Query: universitat pompeu fabra


,Tweet,Rating
0,world progress indian polic govt still tri tak...,0.0
1,kisanektamorcha farmer constantli distroy crop...,0.0
2,reallyswara rohinisgh watch full video,0.0
3,mandeeppunia1 watch full video,0.0
4,mandeeppunia1 watch full video,0.0
5,anoth farmer malkeet singh mahilpur hoshiarpur...,0.0
6,akshaykumar hi tell boss thank,0.0
7,taapse watch full video,0.0
8,despit increas must increas petroleummin pmoindia,0.0
9,mockeri menac charg,0.0


__________________________________________________________________________



### 1. **Query: "protest indian"**
   - The top tweets here are exactly "indian protest," which directly matches the query and has a similarity score of `1.000`. This means they contain both "indian" and "protest," appearing in the most relevant order.
   - As we move down the list, we see variations like "indian farmer protest" or "protest india," which still include both query words but in slightly different contexts.
   - The similarity decreases from `1.000` to `0.929`, suggesting that these tweets are considered slightly less relevant than the first few tweets but remain thematically related.

### 2. **Query: "protest farmer"**
   - The top results here are exclusively "farmer protest," which perfectly matches the query words and has a full similarity score of `1.000`.
   - The next results are variations on the same theme, like "imanikaurrai farmer protest" or "jashanlambi malkitsaggu99 farmer protest," which contain the exact query words but add extra names or words.
   - The similarity remains high across all tweets, around `0.999`, showing they are strongly related to "farmer protest."

### 3. **Query: "support farmer"**
   - All the top 20 tweets are exactly "support farmer," perfectly matching the query with a similarity of `1.000`.
   - There are no additional context variations, which suggests that these exact words are the only highly relevant tweets in the dataset for this query.

### 4. **Query: "free india"**
   - The results for this query are more varied, as there is no exact match for "free india" alone.
   - The top results include tweets with phrases like "british empire arrest freedom fighter today india" and "gujarati kill india," which contain words associated with freedom or the political context in India but in broader contexts.
   - Similarities here are lower, starting at `0.966` and gradually dropping to `0.947`, indicating that while the tweets are somewhat relevant to "free India" or "freedom in India," the match is less direct than in previous queries.

### 5. **Query: "universitat pompeu fabra"**
   - For this query, all results show a similarity of `0.0`. This means there are no tweets in the dataset that contain this phrase or related words.
   - The term "universitat pompeu fabra" seems unrelated to the main themes in the dataset, which appears to focus primarily on protest and political issues in India.

### Overall Conclusion:
- **Direct Match Queries** ("protest indian," "protest farmer," "support farmer"): Word2Vec works well in identifying tweets with exact or near-exact matches, prioritizing tweets that contain the keywords in the same context as the query.
- **Broader Thematic Queries** ("free india"): In these cases, Word2Vec provides tweets generally relevant to the topic but without an exact match. It still finds related contexts, though with slightly lower similarity scores.
- **Queries Not Represented in the Dataset** ("universitat pompeu fabra"): Results with a similarity of `0.0` indicate that no tweets are related to this particular query within the dataset.

This shows that Word2Vec is useful for identifying direct matches and related themes but may struggle if the query topic isn’t represented in the dataset or if the query is very specific and unrelated to the dominant themes in the tweets.

##**2.3. Better than Word2Vec?**

**Can you imagine a better representation than word2vec? Justify your answer.
(HINT - what about Doc2vec? Sentence2vec? Which are the pros and cons?)**

*Yes, there are several representations that build upon or extend the concept of Word2Vec, such as Doc2Vec and Sentence2Vec. Each of these models has specific use cases, strengths, and limitations compared to Word2Vec. Here’s an overview of these representations, their pros, and cons, along with justifications:*

1. **Word2Vec**:
- *Word2Vec represents words as dense, fixed-length vectors based on their context within a corpus. It uses techniques like Continuous Bag of Words (CBOW) and Skip-gram to learn these representations.*
- **Pros**:
  - **Efficient**: *Quick to train and scales well to large datasets.*
  - **Contextual Similarity**: *Captures semantic relationships between words effectively, enabling operations like "king" - "man" + "woman" ≈ "queen".*
- **Cons**:
  - **Lacks Contextual Awareness**: *The same word has one fixed vector, regardless of its meaning in different contexts.*
  - **Sentence and Document Representation**: *Requires post-processing (e.g., averaging word vectors) to create representations for larger text units, which can be crude and lose nuanced meaning.*

2. **Doc2Vec**:
- *Doc2Vec, an extension of Word2Vec, is designed to create dense, fixed-length vector representations of entire documents. It does this by incorporating a unique document ID that acts as an additional vector trained alongside the word vectors.*
- **Pros**:
  - **Captures Document-Level Semantics**: *Represents the global context of a document, making it useful for tasks like document classification, topic modeling, and information retrieval.*
  - **Better than Averaging**: *Unlike simply averaging word vectors, Doc2Vec learns representations that consider the sequence and structure of words in a document.*
- **Cons**:
  - **Training Complexity**: *More complex to train and requires more computational resources compared to Word2Vec.*
  - **Data Dependence**: *The quality of vectors can be heavily dependent on the training corpus, potentially requiring extensive training data for optimal results.*

3. **Sentence2Vec**:
- *Sentence2Vec is used to create vector representations for individual sentences. Approaches vary from simple (e.g., averaging word embeddings) to more sophisticated methods, like using pre-trained models (e.g., Universal Sentence Encoder or BERT).*
- **Pros**:
  - **Sentence-Level Representation**: *Provides vectors that capture the meaning of sentences, which can be more contextually nuanced than Word2Vec or Doc2Vec.*
  - **Use in Similarity and Matching**: *Useful for tasks like sentence similarity, semantic search, and chatbot responses.*
- **Cons**:
  - **Loss of Granularity**: *May overlook fine-grained word-level nuances, especially with simpler methods like averaging.*
  - **Complexity with Contextual Models**: *More advanced models, such as those based on transformers, require substantial computational power and may be overkill for simpler applications.*

**Comparative Justification**:
- **Better Representation?** *Yes, in many cases, Doc2Vec and Sentence2Vec can provide better representations than Word2Vec depending on the task.*
  - **Doc2Vec** *is better for document-level tasks as it captures context more holistically, while Word2Vec's scope is limited to individual words.*
  - **Sentence2Vec** *is beneficial for sentence-level applications where understanding the flow and semantics of sentences is critical. For more complex, context-dependent representations, transformer-based models like **BERT** (Bidirectional Encoder Representations from Transformers) surpass Word2Vec by producing different embeddings for the same word in different contexts, enhancing performance in tasks like question answering and sentiment analysis.*

**Advanced Alternatives**:
- **BERT and Transformer Models**:
  - **Pros**: *Highly contextual embeddings, ability to represent polysemy (same word with different meanings).*
  - **Cons**: *High computational requirements, slower training and inference.*

**Conclusion**:
*The choice between Word2Vec, Doc2Vec, Sentence2Vec, and more advanced models like BERT depends on the specific application:*
- *For word-level tasks, **Word2Vec** is efficient and straightforward.*
- *For document classification or retrieval, **Doc2Vec** offers richer context.*
- *For sentence-level tasks, **Sentence2Vec** or **transformer-based models** like **BERT** provide more nuanced and context-aware embeddings.*

*Overall, Doc2Vec and Sentence2Vec extend the capabilities of Word2Vec by incorporating more complex contexts and structures, making them preferable for applications that require representations beyond isolated words.*